In [1]:
%cd /home/ahol834/resmed202100066-Glaucoma_PRS/andrewholmes2024/summer2025/outcomes/
! pwd

/home/ahol834/resmed202100066-Glaucoma_PRS/andrewholmes2024/summer2025/outcomes
/home/ahol834/resmed202100066-Glaucoma_PRS/andrewholmes2024/summer2025/outcomes


In [2]:
# MUST restart Jupyter before doing below for below to work

import os
os.environ["PYARROW_NUM_THREADS"] = '8'
os.environ["OMP_NUM_THREADS"] = '8'
os.environ["POLARS_MAX_THREADS"] = '16' #8
os.environ["POLARS_NUM_THREADS"] = '16' #8

In [3]:
import numpy as np
import pandas as pd
import miceforest as mf
from joblib import load
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy.stats as stats

import pyarrow as pa 
import pyarrow.csv as pv
import polars as pl

# Load data

In [4]:
def add_cols_for_data_field(data_field, data_cols, cols_to_use_list):
    cols_for_code = data_cols[data_cols.str.startswith(data_field)]
    if len(cols_for_code) < 1:
            print('No columns found matching field ' + data_field)
    cols_to_use_list.extend(cols_for_code)
    return cols_for_code

### ukb49508

In [5]:
# ukb49508 = assesement centre (NOT ocular), inpatient data

ukb49508_csv_cols = pd.read_table('../../../UKBB_Data/ukb49508.tab', nrows=0).columns
ukb49508_cols_to_use = [
    # Demographic
    'f.eid', 
    'f.34.0.0', # Year of birth
    'f.52.0.0', # Month of brith
    'f.53.0.0', # date attending assesement centre
    'f.21003.0.0', # age at assesement centre
    'f.31.0.0', # sex
    'f.22001.0.0', # genetic sex

    'f.21000.0.0', # Ethnicity
    
] 

# pregnant
add_cols_for_data_field('f.3140', ukb49508_csv_cols, ukb49508_cols_to_use) # pregnant

# Self-reported conditions
f20002_cols = add_cols_for_data_field('f.20002', ukb49508_csv_cols, ukb49508_cols_to_use) # self-report conditions
f20009_cols = add_cols_for_data_field('f.20009', ukb49508_csv_cols, ukb49508_cols_to_use) # interpolated age self-reported medical conditions diagnosed
f6148_cols = add_cols_for_data_field('f.6148', ukb49508_csv_cols, ukb49508_cols_to_use) # eye conditions
add_cols_for_data_field('f.4689', ukb49508_csv_cols, ukb49508_cols_to_use) # age glaucoma dx
add_cols_for_data_field('f.5923', ukb49508_csv_cols, ukb49508_cols_to_use) # age AMD dx
add_cols_for_data_field('f.4700', ukb49508_csv_cols, ukb49508_cols_to_use) # age cat dx
add_cols_for_data_field('f.5901', ukb49508_csv_cols, ukb49508_cols_to_use) # age DR dx

# Surgeries
f5326_cols = add_cols_for_data_field('f.5326', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported surgery
f5327_cols = add_cols_for_data_field('f.5327', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported laser tx
f20004_cols = add_cols_for_data_field('f.20004', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported surgery
f20010_cols = add_cols_for_data_field('f.20010', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported surgery interpolayed year
f20011_cols = add_cols_for_data_field('f.20011', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported surgery interpolayed age
f20003_cols = add_cols_for_data_field('f.20003', ukb49508_csv_cols, ukb49508_cols_to_use) # self-reported medication
add_cols_for_data_field('f.5324', ukb49508_csv_cols, ukb49508_cols_to_use) # cataract surgery

# Inpatient
ICD10_inpatient_dx_cols  = add_cols_for_data_field('f.41270.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD10 inpatient diagnosis 
ICD10_inpatient_date_cols = add_cols_for_data_field('f.41280.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD10 inpatient date

ICD9_inpatient_dx_cols = add_cols_for_data_field('f.41271.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD9 inpatient diagnosis 
ICD9_inpatient_date_cols = add_cols_for_data_field('f.41281.0', ukb49508_csv_cols, ukb49508_cols_to_use) # ICD9 inpatient date

OPCS4_inpatient_operation_cols = add_cols_for_data_field('f.41272.0', ukb49508_csv_cols, ukb49508_cols_to_use) # OPCS4 inpatient operation code
OPCS4_inpatient_operation_date_cols = add_cols_for_data_field('f.41282.0', ukb49508_csv_cols, ukb49508_cols_to_use) # OPCS4 inpatient operation code

OPCS3_inpatient_operation_cols = add_cols_for_data_field('f.41273.0', ukb49508_csv_cols, ukb49508_cols_to_use) # OPCS3 inpatient operation code
OPCS3_inpatient_operation_date_cols = add_cols_for_data_field('f.41283.0', ukb49508_csv_cols, ukb49508_cols_to_use) # OPCS3 inpatient operation code

ukb49508_cols_to_use = np.unique(ukb49508_cols_to_use)
print(f'Found {len(ukb49508_cols_to_use)} columns')

Found 1728 columns


In [6]:
date_cols = (
    ['f.53.0.0']
    + list(ICD10_inpatient_date_cols)
    + list(ICD9_inpatient_date_cols)
    + list(OPCS4_inpatient_operation_date_cols)
    + list(OPCS3_inpatient_operation_date_cols)
)

# Build dtype mapping for str diagnosis columns
dtype_dict = {col: 'string' for col in (
    list(f20003_cols)
    + list(ICD10_inpatient_dx_cols) + list(ICD9_inpatient_dx_cols)
    + list(OPCS4_inpatient_operation_cols) + list(OPCS3_inpatient_operation_cols)
)}


In [7]:
%%time

# UNCOMMENT desired method of loading data below:
# ##################################

# ### (A) LOAD FROM PKL FILE (FASTER)
# ##################################
# ukb49508_df = pd.read_pickle('./data/extracted_raw_ukb49508.pkl')


# ------------
# OR
# ------------


# ### (B)LOAD FROM RAW FILE - if .pkl not already made
# ##################################
dtype_dict_dates = {c: "string" for c in date_cols}
dtype_dict_all = {**dtype_dict, **dtype_dict_dates}
ukb49508_df = pd.read_table(
    '../../../UKBB_Data/ukb49508.tab',
    usecols=ukb49508_cols_to_use,
    engine='c', # DO NOT use pyarrow
    date_format='%Y-%m-%d',
    dtype=dtype_dict_all,
    low_memory=False,
    memory_map=True,
)
# ## Save a raw copy .pkl - if loading from raw file
# ukb49508_df.to_pickle('./data/extracted_raw_ukb49508.pkl', compression=None)

CPU times: user 3min 20s, sys: 2min 54s, total: 6min 15s
Wall time: 6min 15s


In [8]:
## Ensure date columns fixed - MUST RUN
ukb49508_df[date_cols] = ukb49508_df[date_cols].apply(
    pd.to_datetime,
    errors="coerce",
    format="%Y-%m-%d",
)


## Other files

In [9]:
%%time 

# Ocular data

ukb51745_df = pd.read_table(
    '../../../UKBB_Data_Basket3/ukb51745.tab', 
    usecols=['f.eid', 'f.5262.0.0', 'f.5254.0.0'],
    engine='c',
    low_memory=False,
    memory_map=True,
)

# Align indices
ukb51745_df = ukb49508_df[['f.eid']].merge(ukb51745_df, on='f.eid', how='left')

CPU times: user 12.3 s, sys: 10.4 s, total: 22.7 s
Wall time: 22.8 s


In [10]:
%%time

# Add deprivation index

ukb675501_df = pd.read_table(
    '../../../UKBB_Data_healthrelatedoutcomes/ukb675501.tab', 
    usecols=['f.eid', 'f.22189.0.0'],
    engine='c',
    low_memory=False,
    memory_map=True,
)

# Align indices
ukb675501_df = ukb49508_df[['f.eid']].merge(ukb675501_df, on='f.eid', how='left')

CPU times: user 6.52 s, sys: 6.09 s, total: 12.6 s
Wall time: 12.9 s


In [11]:
# GP
# n.b. already has f.53.0.0 merged in

merged_gp_events_df = pd.read_pickle('./data/raw_gp_events_merged_dates.pkl')
merged_gp_prescriptions_df = pd.read_pickle('./data/raw_gp_prescriptions_merged_dates.pkl')

## coerce dates
merged_gp_events_df[['event_dt', 'f.53.0.0']] = merged_gp_events_df[["event_dt", "f.53.0.0"]].apply(pd.to_datetime, errors="coerce", format="%Y-%m-%d", cache=True)
merged_gp_prescriptions_df[['issue_date', 'f.53.0.0']] = merged_gp_prescriptions_df[["issue_date", "f.53.0.0"]].apply(pd.to_datetime, errors="coerce", format="%Y-%m-%d", cache=True)

# coerece str
merged_gp_events_df['read_2'] = merged_gp_events_df['read_2'].astype("string")
merged_gp_events_df['read_3'] = merged_gp_events_df['read_3'].astype("string")
merged_gp_prescriptions_df['bnf_code'] = merged_gp_prescriptions_df['bnf_code'].astype("string")

## Init

In [21]:
diagnosis_df = pd.DataFrame()
diagnosis_df['f.eid'] = ukb49508_df['f.eid']

In [22]:
est_dob = pd.to_datetime(dict(year=ukb49508_df['f.34.0.0'], month=ukb49508_df['f.52.0.0'], day=15))
diagnosis_df['DOB_est'] = est_dob

In [23]:
#### Minor exclusions

cols_3140 = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.3140')]
diagnosis_df['Ever pregnant'] = ukb49508_df[cols_3140].eq(1).any(axis=1)

diagnosis_df['Sex_nondisc'] = np.select(
    [
        ukb49508_df['f.22001.0.0'].isna() & ukb49508_df['f.31.0.0'].isna(),
        ukb49508_df['f.22001.0.0'].isna() & ukb49508_df['f.31.0.0'].notna(),
        ukb49508_df['f.22001.0.0'].notna() & ukb49508_df['f.31.0.0'].isna(),
        ukb49508_df['f.22001.0.0'] == ukb49508_df['f.31.0.0'],
        ukb49508_df['f.22001.0.0'] != ukb49508_df['f.31.0.0'],
    ],
    [
        np.nan,
        ukb49508_df['f.31.0.0'],
        ukb49508_df['f.22001.0.0'],
        ukb49508_df['f.31.0.0'],
        np.nan,
    ],
    default=np.nan
)

# Functions

In [12]:
def extract_gp_events(
    events_df, 
    read_v2_codes, 
    read_v3_codes, 
    baseline_feid_set, 
    incident_feid_set, 
    invalid_feid_set,
    invalid_gp_dates=None,
):
    if invalid_gp_dates is None:
        invalid_gp_dates = [
            pd.to_datetime('01/01/1901', format='%d/%m/%Y'), # precedes DOB
            pd.to_datetime('02/02/1902', format='%d/%m/%Y'), # matches DOB
            pd.to_datetime('03/03/1903', format='%d/%m/%Y'), # follows DOB, but is in year of birth
            pd.to_datetime('07/07/2037', format='%d/%m/%Y') # dated in the future meaning system default / placeholder
        ]
            
    # All cases
    any_dx = (events_df['read_2'].isin(read_v2_codes)) | (events_df['read_3'].isin(read_v3_codes))
    any_dx_invalid = any_dx & (events_df['event_dt'].isnull() | events_df['event_dt'].isin(invalid_gp_dates))

    # Baseline cases
    any_baseline_dx = any_dx & (events_df['event_dt'] <= events_df['f.53.0.0'])
    baseline_dx_valid = any_baseline_dx & ~any_dx_invalid
    baseline_feid_set.update(events_df[baseline_dx_valid]['f.eid'])

    # Incident cases
    any_incident_dx = any_dx & (events_df['event_dt'] > events_df['f.53.0.0'])
    incident_dx_valid = any_incident_dx & ~any_dx_invalid
    incident_feid_set.update(events_df[incident_dx_valid]['f.eid'])

    # Earliest date of GP diagnosis
    earliest_date_gp = events_df[baseline_dx_valid | incident_dx_valid].groupby('f.eid')['event_dt'].min()

    # Invalid date counts
    n_invalid = any_dx_invalid.sum()
    invalid_feid_set.update(events_df[any_dx_invalid]['f.eid'])

    print(f'Participants with invalid age for Dx in GP records: {n_invalid}')

    return earliest_date_gp, n_invalid

In [13]:
def extract_gp_prescriptions(
    rx_df, 
    bnf_codes, 
    baseline_feid_set, 
    incident_feid_set,
    invalid_gp_dates=None,
):
    if invalid_gp_dates is None:
        invalid_gp_dates = [
            pd.to_datetime('01/01/1901', format='%d/%m/%Y'), # precedes DOB
            pd.to_datetime('02/02/1902', format='%d/%m/%Y'), # matches DOB
            pd.to_datetime('03/03/1903', format='%d/%m/%Y'), # follows DOB, but is in year of birth
            pd.to_datetime('07/07/2037', format='%d/%m/%Y') # dated in the future meaning system default / placeholder
        ]
    
    # All cases
    any_med = rx_df['bnf_code'].isin(bnf_codes)
    any_med_valid = any_med & ~(rx_df['issue_date'].isin(invalid_gp_dates))

    # Baseline 
    baseline_glaucoma_med = any_med_valid & (rx_df['issue_date'] <= rx_df['f.53.0.0'])
    baseline_valid = baseline_glaucoma_med & ~(rx_df['issue_date'].isin(invalid_gp_dates))
    baseline_feid_set.update(rx_df[baseline_valid]['f.eid'])
    
    # Incident 
    incident_glaucoma_med = any_med_valid & (rx_df['issue_date'] > rx_df['f.53.0.0'])
    incident_valid = incident_glaucoma_med & ~(rx_df['issue_date'].isin(invalid_gp_dates))
    incident_feid_set.update(rx_df[incident_valid]['f.eid'])

    # Earliest date of GP presc
    earliest_date_gp = rx_df[baseline_valid | incident_valid].groupby('f.eid')['issue_date'].min()

    # Invalid date counts
    n_baseline_invalid = baseline_glaucoma_med.sum() - baseline_valid.sum()
    n_incident_invalid = incident_glaucoma_med.sum() - incident_valid.sum()

      
    print(f'Participants with invalid age for Dx in GP records: {n_baseline_invalid + n_incident_invalid}')

    return earliest_date_gp, n_baseline_invalid, n_incident_invalid

In [14]:
# Process inpatient matrix (operations or diagnoses)
# (no invalid dates)
def process_inpatient_matrix(df, diagnosis_col_list, date_col_list, positive_values_list, baseline_feid_set, incident_feid_set, feid_dates_df=None):
    for col_idx in range(len(diagnosis_col_list)):
        dx_col = diagnosis_col_list[col_idx]
        date_col = date_col_list[col_idx]
        
        any_dx = df[dx_col].isin(positive_values_list) 
        baseline_dx = any_dx & (df[date_col] <= df['f.53.0.0'])
        incident_dx = any_dx & (df[date_col] > df['f.53.0.0'])

        baseline_feid_set.update(df[baseline_dx]['f.eid'])
        incident_feid_set.update(df[incident_dx]['f.eid'])

        # Add to first inpatient dict for keeping track of earliest diagnoses
        # NOTE: may have multiple instances: to find earliest instance, group by f.eid and get min()
        if feid_dates_df is not None:
            for index, row in df[['f.eid', date_col]][any_dx].iterrows():
                feid_dates_df.loc[len(feid_dates_df)] = [row['f.eid'], row[date_col]]

In [15]:
def extract_inpatient_diagnoses(df, icd10_codes, icd9_codes, baseline_feid_set, incident_feid_set):
    ICD10_inpatient_dx_cols = df.columns[df.columns.str.startswith('f.41270.0')]
    ICD10_inpatient_date_cols = df.columns[df.columns.str.startswith('f.41280.0')]
    
    ICD9_inpatient_dx_cols = df.columns[df.columns.str.startswith('f.41271.0')]
    ICD9_inpatient_date_cols = df.columns[df.columns.str.startswith('f.41281.0')]

    feid_dates_df = pd.DataFrame(columns=['f.eid', 'earliest_inpatient_diagnosis_date'])
    
    # Extract ICD10
    if len(icd10_codes) > 0:
        process_inpatient_matrix(
            df,
            ICD10_inpatient_dx_cols,
            ICD10_inpatient_date_cols,
            icd10_codes,
            baseline_feid_set,
            incident_feid_set,
            feid_dates_df
        )

    # Extract ICD9
    if len(icd9_codes) > 0:
        process_inpatient_matrix(
            df,
            ICD9_inpatient_dx_cols,
            ICD9_inpatient_date_cols,
            icd9_codes,
            baseline_feid_set,
            incident_feid_set,
            feid_dates_df
        )

    # Return earliest date of inpatient diagnosis
    return feid_dates_df.groupby('f.eid')['earliest_inpatient_diagnosis_date'].min()


In [16]:
def process_self_reported_eye_disease(input_df, positive_vals_f6148, age_diagnosed_field, positive_vals_f20002, output_df, outcome_col, outcome_age_col):
    invalid_ages = [-3, -1]
    
    # Init
    output_df[outcome_col] = np.nan
    output_df[outcome_age_col] = np.nan

    baseline_diagnosis_from_6148 = np.zeros(len(input_df), dtype=bool)
    incident_diagnosis_from_6148 = np.zeros(len(input_df), dtype=bool)
    dx_any_0 = np.zeros(len(input_df), dtype=bool)
    dx_any_inc = np.zeros(len(input_df), dtype=bool)

    ###### Field 6148
    if positive_vals_f6148 is not None:
        # 6148 baseline
        cols_for_6148_0 = input_df.columns[input_df.columns.str.startswith('f.6148.0')]
        baseline_diagnosis_from_6148 = input_df[cols_for_6148_0].isin(positive_vals_f6148).any(axis=1)
        
        # 6148 incident (instances 1–3)
        cols_for_6148_inc = np.concatenate([
            input_df.columns[input_df.columns.str.startswith('f.6148.1')],
            input_df.columns[input_df.columns.str.startswith('f.6148.2')],
            input_df.columns[input_df.columns.str.startswith('f.6148.3')],
        ])
        incident_diagnosis_from_6148 = input_df[cols_for_6148_inc].isin(positive_vals_f6148).any(axis=1)
        
        ##### Baseline age from f age_diagnosed_field
        if age_diagnosed_field is not None:
            age_baseline = input_df[f'f.{age_diagnosed_field}.0.0']
            output_df.loc[baseline_diagnosis_from_6148, outcome_age_col] = age_baseline[baseline_diagnosis_from_6148]
        
            age_invalid = output_df[outcome_age_col].isin(invalid_ages)
            output_df.loc[age_invalid, outcome_age_col] = np.nan
    
            age_na_and_dx = baseline_diagnosis_from_6148 & input_df[f'f.{age_diagnosed_field}.0.0'].isna()
    
            print('Baseline')
            print(f'Participants with invalid age in f{age_diagnosed_field}: {age_invalid.sum()}')
            print(f'Participants with disease in f6148, but NA age diagnosed in f{age_diagnosed_field}: {age_na_and_dx.sum()}')
    
        ##### Incident age from f age_diagnosed_field
        if age_diagnosed_field is not None:
            incident_age_cols = [f'f.{age_diagnosed_field}.1.0', f'f.{age_diagnosed_field}.2.0', f'f.{age_diagnosed_field}.3.0']
            age_incident = input_df[incident_age_cols].min(axis=1, skipna=True)
            output_df.loc[incident_diagnosis_from_6148, outcome_age_col] = age_incident[incident_diagnosis_from_6148]
        
            age_invalid = output_df[outcome_age_col].isin(invalid_ages)
            output_df.loc[age_invalid, outcome_age_col] = np.nan
        
            print('Incident')
            print(f'Participants with invalid age in f{age_diagnosed_field}: {age_invalid.sum()}')

    ###### Baseline, f20002 & 20009
    if positive_vals_f20002 is not None:
        cols_20002_0 = input_df.columns[input_df.columns.str.startswith('f.20002.0')]
        age_cols_20009_0 = input_df.columns[input_df.columns.str.startswith('f.20009.0')]
        dx_mat_0 = input_df[cols_20002_0].isin(positive_vals_f20002)
        dx_any_0 = dx_mat_0.any(axis=1)
        age_mat_0 = input_df[age_cols_20009_0].astype(float)
        invalid_mask_0 = age_mat_0.isin(invalid_ages)
        age_mat_0 = age_mat_0.mask(invalid_mask_0)
        dx_for_age_0 = dx_mat_0.copy()
        dx_for_age_0.columns = age_mat_0.columns
        ages_where_dx_0 = age_mat_0.where(dx_for_age_0)
        baseline_age_20002 = ages_where_dx_0.min(axis=1, skipna=True) #ages_where_glaucoma_0.bfill(axis=1).iloc[:, 0]
        row_has_valid_0 = baseline_age_20002.notna()
    
        output_df.loc[row_has_valid_0, outcome_age_col] = baseline_age_20002[row_has_valid_0]
    
        n_invalid_age_baseline = (invalid_mask_0 & dx_for_age_0).sum().sum()
        print(f'Baseline, participants with invalid age in f20009.0: {n_invalid_age_baseline}')
    
        ###### Incident, f20002 & 20009
        cols_20002_inc = np.concatenate([
            input_df.columns[input_df.columns.str.startswith('f.20002.1')],
            input_df.columns[input_df.columns.str.startswith('f.20002.2')],
            input_df.columns[input_df.columns.str.startswith('f.20002.3')],
        ])
        age_cols_20009_inc = np.concatenate([
            input_df.columns[input_df.columns.str.startswith('f.20009.1')],
            input_df.columns[input_df.columns.str.startswith('f.20009.2')],
            input_df.columns[input_df.columns.str.startswith('f.20009.3')],
        ])
        dx_mat_inc = input_df[cols_20002_inc].isin(positive_vals_f20002)
        dx_any_inc = dx_mat_inc.any(axis=1)
        age_mat_inc = input_df[age_cols_20009_inc].astype(float)
        invalid_mask_inc = age_mat_inc.isin(invalid_ages)
        age_mat_inc = age_mat_inc.mask(invalid_mask_inc)
        dx_for_age_inc = dx_mat_inc.copy()
        dx_for_age_inc.columns = age_mat_inc.columns
        ages_where_dx_inc = age_mat_inc.where(dx_for_age_inc)
        incident_age_20002 = ages_where_dx_inc.min(axis=1, skipna=True) #ages_where_glaucoma_0.bfill(axis=1).iloc[:, 0]
        row_has_valid_inc = incident_age_20002.notna()
    
        output_df.loc[row_has_valid_inc, outcome_age_col] = incident_age_20002[row_has_valid_inc]
    
        n_invalid_age_incident = (invalid_mask_inc & dx_for_age_inc).sum().sum()
        print(f'Incident, participants with invalid age in f20009.0: {n_invalid_age_incident}')

    ###### Set values
    baseline_mask = baseline_diagnosis_from_6148 | dx_any_0
    incident_mask = (incident_diagnosis_from_6148 | dx_any_inc) & ~baseline_mask
    labels = np.full(len(output_df), 'Undiagnosed', dtype=object)
    labels[incident_mask] = 'Incident'
    labels[baseline_mask] = 'Baseline'
    output_df[outcome_col] = labels

In [17]:
def extract_disease_outcomes(input_df, output_df, col_name, self_report_data, gp_data, inpatient_data, skip_final_definitions=False):
    feid_col = output_df['f.eid']

    # Column names
    sr_dx_col = f'{col_name}_self_reported'
    sr_age_col = f'{col_name}_earliest_age_diagnosed_self_report'
    
    gp_dx_col = f'{col_name}_GP_diagnosis'
    gp_date_col = f'{col_name}_earliest_GP_date'
    gp_age_col = f'{col_name}_earliest_age_diagnosed_GP'
    
    ip_dx_col = f'{col_name}_inpatient_diagnosis'
    ip_date_col =  f'{col_name}_earliest_inpatient_date'
    ip_age_col =  f'{col_name}_earliest_age_diagnosed_inpatient'
    
    # Clear previous (avoid bugs)
    output_df.drop(columns=[
        sr_dx_col, sr_age_col,
        gp_dx_col, gp_date_col, gp_age_col,
        ip_dx_col, ip_date_col, ip_age_col
    ], inplace=True, errors='ignore')
    
    #### Self-reported
    print('\nSelf-report')

    process_self_reported_eye_disease(
        input_df = input_df,
        output_df = output_df,
        outcome_col = sr_dx_col,
        outcome_age_col = sr_age_col,
        **self_report_data
    )
    print(output_df[sr_dx_col].value_counts())
    print(output_df[sr_age_col].describe())

    #### GP
    ###################################
    baseline_gp_feids = set()
    incident_gp_feids = set()
    invalid_gp_feids = set()

    print('\nGP')
    earliest_date_gp, n_invalid = extract_gp_events(
        events_df = merged_gp_events_df,
        baseline_feid_set = baseline_gp_feids,
        incident_feid_set = incident_gp_feids,
        invalid_feid_set = invalid_gp_feids,
        **gp_data
    )
    
    # Generate masks
    baseline_dx = feid_col.isin(baseline_gp_feids)
    incident_dx = feid_col.isin(incident_gp_feids)
    invalid_dx = feid_col.isin(invalid_gp_feids)
    
    # Set columns
    output_df[gp_dx_col] = 'Undiagnosed'
    output_df.loc[invalid_dx, gp_dx_col] = 'Invalid'
    output_df.loc[incident_dx, gp_dx_col] = 'Incident'
    output_df.loc[baseline_dx, gp_dx_col] = 'Baseline'

    #output_df.loc[feid_col.isin(earliest_date_gp.index), gp_date_col] = earliest_date_gp.values
    output_df[gp_date_col] = pd.to_datetime(output_df["f.eid"].map(earliest_date_gp), errors="coerce")

    # Add earliest age of GP diagnosis
    # Only month and year of birth available: so DOB estimated with day as 1
    valid_mask = output_df[gp_date_col].notna() & output_df['DOB_est'].notna()
    delta_days = (output_df.loc[valid_mask, gp_date_col] - output_df.loc[valid_mask, 'DOB_est']).dt.days
    output_df.loc[valid_mask, gp_age_col] = np.floor(delta_days / 365.25)

    print(output_df[gp_dx_col].value_counts())
    print(output_df[gp_date_col].describe())
    print(output_df[gp_age_col].describe())

    #### Inpatient
    ###################################
    baseline_inpatient_feids = set()
    incident_inpatient_feids = set()

    print('\nInpatient')
    earliest_date_inpatient = extract_inpatient_diagnoses(
        df = input_df,
        baseline_feid_set = baseline_inpatient_feids,
        incident_feid_set = incident_inpatient_feids,
        **inpatient_data
    )

    # Generate masks
    baseline_dx = feid_col.isin(baseline_inpatient_feids)
    incident_dx = feid_col.isin(incident_inpatient_feids)
    
    # Set columns
    # Set "baseline" second as this has higher priority (cannot be incident if also was baseline)
    output_df[ip_dx_col] = 'Undiagnosed'
    output_df.loc[incident_dx, ip_dx_col] = 'Incident'
    output_df.loc[baseline_dx, ip_dx_col] = 'Baseline'

    output_df.loc[feid_col.isin(earliest_date_inpatient.index), ip_date_col] = earliest_date_inpatient.values

    # Add earliest age of inpatient diagnosis
    # Only month and year of birth available: so DOB estimated with day as 1
    valid_mask = output_df[ip_date_col].notna() & output_df['DOB_est'].notna()
    delta_days = (output_df.loc[valid_mask, ip_date_col] - output_df.loc[valid_mask, 'DOB_est']).dt.days
    output_df.loc[valid_mask, ip_age_col] = np.floor(delta_days / 365.25)

    print(output_df[ip_dx_col].value_counts())
    print(output_df[ip_date_col].describe())
    print(output_df[ip_age_col].describe())

    #### Pool columns
    ###################################
    print('\nPooled')

    # Diagnosis
    
    incident_self_report = output_df[sr_dx_col].eq('Incident')
    incident_GP          = output_df[gp_dx_col].eq('Incident')
    incident_inpatient   = output_df[ip_dx_col].eq('Incident')
    any_incident_dx      = incident_self_report | incident_GP | incident_inpatient
    
    baseline_self_report = output_df[sr_dx_col].eq('Baseline')
    baseline_GP          = output_df[gp_dx_col].eq('Baseline')
    baseline_inpatient   = output_df[ip_dx_col].eq('Baseline')
    any_baseline_dx      = baseline_self_report | baseline_GP | baseline_inpatient
    
    # Baseline overrides incident if both
    any_dx = any_baseline_dx | any_incident_dx
    baseline_mask = any_baseline_dx
    incident_mask = any_incident_dx & ~baseline_mask
    
    labels = np.full(len(output_df), 'Control', dtype=object)
    labels[incident_mask] = 'Incident'
    labels[baseline_mask] = 'Baseline'

    pooled_dx_col = f'{col_name}_diagnosis_type'
    output_df[pooled_dx_col] = labels

    # Earliest age column
    ## (Does not include surgically defined individuals, if any)
    pooled_age_col = f'{col_name}_earliest_age_diagnosed'
    output_df[pooled_age_col] = output_df[[sr_age_col, gp_age_col, ip_age_col]].min(axis=1, skipna=True)

    print(output_df[pooled_dx_col].value_counts())
    print(output_df[pooled_age_col].describe())

    #### Definitions with exclusions
    ###################################

    if skip_final_definitions:
        print('\nSkipping final definitions')
        return pooled_dx_col, pooled_age_col, sr_dx_col, sr_age_col, gp_dx_col, gp_date_col, gp_age_col, ip_dx_col, ip_date_col, ip_age_col

    print('\nFinal definitions')
    # Exclusions for all
    ever_pregnant = output_df['Ever pregnant'] == 1
    discord_sex = output_df['Sex_nondisc'].isna()

    # invalid GP date with no other dx source
    # Include as prevalent case, but exclude from study baseline/incident
    invalid_gp_mask = output_df[gp_dx_col] == 'Invalid'
    invalid_gp_no_other_dx = invalid_gp_mask & (output_df[pooled_dx_col] == 'Control')

    #### Prevalence
    prevalence_col = f'Prevalent {col_name} minor exclusions'
    output_df[prevalence_col] = output_df[pooled_dx_col].isin(['Baseline', 'Incident']).astype(int)
    output_df.loc[invalid_gp_no_other_dx, prevalence_col] = 1 # GP invalid date issue
    output_df.loc[ever_pregnant | discord_sex, prevalence_col] = np.nan # minor exclusions
    print(f'\n{prevalence_col}')
    print(f'{output_df[prevalence_col].value_counts()}')
    print(f'NA: {output_df[prevalence_col].isna().sum()}')

    ### Incident
    incident_col = f'Incident {col_name} minor exclusions'
    output_df[incident_col] = output_df[pooled_dx_col].eq('Incident').astype(int)
    output_df.loc[invalid_gp_no_other_dx, incident_col] = np.nan # GP invalid date issue
    output_df.loc[ever_pregnant | discord_sex, incident_col] = np.nan # minor exclusions
    output_df.loc[output_df[pooled_dx_col] == 'Baseline', incident_col] = np.nan # exclude baseline dx
    print(f'\n{incident_col}')
    print(f'{output_df[incident_col].value_counts()}')
    print(f'NA: {output_df[incident_col].isna().sum()}')

    ### Baseline
    baseline_col = f'Baseline {col_name} minor exclusions'
    output_df[baseline_col] = output_df[pooled_dx_col].eq('Baseline').astype(int)
    output_df.loc[invalid_gp_no_other_dx, baseline_col] = np.nan # GP invalid date issue
    output_df.loc[ever_pregnant | discord_sex, baseline_col] = np.nan # minor exclusions
    output_df.loc[output_df[pooled_dx_col] == 'Incident', baseline_col] = np.nan # exclude incident dx
    print(f'\n{baseline_col}')
    print(f'{output_df[baseline_col].value_counts()}')
    print(f'NA: {output_df[baseline_col].isna().sum()}')

    return pooled_dx_col, pooled_age_col, sr_dx_col, sr_age_col, gp_dx_col, gp_date_col, gp_age_col, ip_dx_col, ip_date_col, ip_age_col

# Glaucoma surgeries and medications (for glaucoma definition control exclusions)
- for glaucoma definition control exclusions (not surgery outcomes); includes laser, etc

In [24]:
# Init column

diagnosis_df['prevalent_glaucoma_surgery'] = 0
diagnosis_df['prevalent_glaucoma_med'] = 0
diagnosis_df['prevalent_glaucoma_surgery_or_med'] = 0

diagnosis_df['baseline_glaucoma_surgery'] = 0
diagnosis_df['baseline_glaucoma_med'] = 0
diagnosis_df['baseline_glaucoma_surgery_or_med'] = 0

diagnosis_df['incident_glaucoma_surgery'] = 0
diagnosis_df['incident_glaucoma_med'] = 0
diagnosis_df['incident_glaucoma_surgery_or_med'] = 0

### Self-reported

In [25]:
self_report_meds_codes = ["1141190854","1141150754","1140875934","1140921818","1141150750","1140875816","1140869452","1140883574","1140883566","1140883550","1140875508","1140875828","1140878080","1140878092","1140927928","1140853558","1140875868","1140875870","1140856568","1140922714","1141176284","1141176288","1140922718","1140879866","1140879758","1140879826","1140875808","1140879822","1140878098","1141169516","1140875840","1141169520","1141184726","1140881882","1140877988","1140881890","1141179734","1140877994","1141146188","1141179914","1141185316","1141146198","1141179920","1141185326","1141184722","1140853574","1140853596","1140864342","1140877992","1140878020","1140878212","1140879828","1140888536","1141145646"]

In [26]:
### Baseline

# Self-reported surgeries

# 5326 ever had surgery for glaucoma or OHT
positive_vals_5326 = [2, 3, 4]
baseline_surgery_from_5326 = ukb49508_df['f.5326.0.0'].isin(positive_vals_5326)

# 5327 ever had laser tx for glaucoma or OHT
positive_vals_5327 = [2, 3, 4]
baseline_surgery_from_5327 = ukb49508_df['f.5327.0.0'].isin(positive_vals_5327)

# 1436 (glaucoma surgery / trabeculectomy) in 20004
baseline_surgery_20004_feids = set()
cols_20004 = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.0')]
for col_idx in range(len(cols_20004)): 
    col = cols_20004[col_idx]
    surgery = ukb49508_df[col].eq(1436)
    baseline_surgery_20004_feids.update(ukb49508_df[surgery]['f.eid'])

baseline_surgery_from_20004 = ukb49508_df['f.eid'].isin(baseline_surgery_20004_feids)

# Meds in 20003
cols_20003 = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20003.0')]
baseline_meds_from_20003 = ukb49508_df[cols_20003].isin(self_report_meds_codes).any(axis=1)

# Set column values

diagnosis_df.loc[baseline_surgery_from_5326 | baseline_surgery_from_5327 | baseline_surgery_from_20004, 'baseline_glaucoma_surgery'] = 1
diagnosis_df.loc[baseline_meds_from_20003, 'baseline_glaucoma_med'] = 1

In [27]:
### Incident

# Self-reported surgeries

# 5326 ever had surgery for glaucoma or OHT
cols_5326_inc = np.concatenate([
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5326.1')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5326.2')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5326.3')],
])
positive_vals_5326 = [2, 3, 4]
incident_surgery_from_5326 = ukb49508_df[cols_5326_inc].isin(positive_vals_5326).any(axis=1)

# 5327 ever had laser tx for glaucoma or OHT
cols_5327_inc = np.concatenate([
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5327.1')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5327.2')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5327.3')],
])
positive_vals_5327 = [2, 3, 4]
incident_surgery_from_5327 = ukb49508_df[cols_5327_inc].isin(positive_vals_5327).any(axis=1)

# 1436 (glaucoma surgery / trabeculectomy) in 20004
incident_surgery_20004_feids = set()
cols_20004_inc = np.concatenate([
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.1')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.2')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.3')],
])
for col_idx in range(len(cols_20004_inc)): 
    col = cols_20004_inc[col_idx]
    surgery = ukb49508_df[col].eq(1436)
    incident_surgery_20004_feids.update(ukb49508_df[surgery]['f.eid'])

incident_surgery_from_20004 = ukb49508_df['f.eid'].isin(incident_surgery_20004_feids)

# Meds in 20003
cols_20003_inc = np.concatenate([
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20003.1')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20003.2')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20003.3')],
])
incident_meds_from_20003 = ukb49508_df[cols_20003_inc].isin(self_report_meds_codes).any(axis=1)

# Set column values

diagnosis_df.loc[incident_surgery_from_5326 | incident_surgery_from_5327 | incident_surgery_from_20004, 'incident_glaucoma_surgery'] = 1
diagnosis_df.loc[incident_meds_from_20003, 'incident_glaucoma_med'] = 1

### GP

In [28]:
# Research group GP operation codes
surgery_read_v2codes = ["72550","72554","72554","72560","72561","72562","72562","72563","72563","72564","72603","72605"]
surgery_read_v3codes = ["72550","72550","72550","72554","72554","72554","72560","72560","72560","72561","72562","72562","72562","72563","72563","72563","72603","X00Wr","X00Ws","X00XI","XE0BN","XE0BO","XE0BP","XaEXv","XaJdX","XaJhf","XaKPD","XaKaz","XaKb1","XaKb3","XaXCJ"]

In [29]:
baseline_gp_feids = set()
incident_gp_feids = set()
invalid_gp_feids = set()

earliest_date_gp, n_invalid = extract_gp_events(merged_gp_events_df, surgery_read_v2codes, surgery_read_v3codes, baseline_gp_feids, incident_gp_feids, invalid_gp_feids)
print(f'Earliest GP diagnosis dates: \n{earliest_date_gp.describe()}')

Participants with invalid age for Dx in GP records: 7
Earliest GP diagnosis dates: 
count                              616
mean     2007-08-21 13:59:13.246753024
min                1971-07-05 00:00:00
25%                2003-08-29 12:00:00
50%                2010-12-07 00:00:00
75%                2014-03-11 18:00:00
max                2017-04-12 00:00:00
Name: event_dt, dtype: object


In [30]:
# Set column values

baseline_surgery_from_gp = diagnosis_df['f.eid'].isin(baseline_gp_feids)
diagnosis_df.loc[baseline_surgery_from_gp, 'baseline_glaucoma_surgery'] = 1

incident_surgery_from_gp = diagnosis_df['f.eid'].isin(incident_gp_feids)
diagnosis_df.loc[incident_surgery_from_gp, 'incident_glaucoma_surgery'] = 1

In [31]:
#### for testing invalid

# Generate masks
baseline_dx = diagnosis_df['f.eid'].isin(baseline_gp_feids)
incident_dx = diagnosis_df['f.eid'].isin(incident_gp_feids)
invalid_dx = diagnosis_df['f.eid'].isin(invalid_gp_feids)

# Set columns
diagnosis_df['GP_glaucoma_surgery'] = 'Undiagnosed'
diagnosis_df.loc[invalid_dx, 'GP_glaucoma_surgery'] = 'Invalid'  # set first. ihas another date valid then is not invalid
diagnosis_df.loc[incident_dx, 'GP_glaucoma_surgery'] = 'Incident'
diagnosis_df.loc[baseline_dx, 'GP_glaucoma_surgery'] = 'Baseline'

diagnosis_df['GP_glaucoma_surgery'].value_counts()

GP_glaucoma_surgery
Undiagnosed    501800
Incident          349
Baseline          267
Invalid             3
Name: count, dtype: int64

In [32]:
# Prescriptions

# Research group GP medication codes
gp_bnf_med_codes = pd.read_table('./codes/gp_scripts_bnfglaucoma.txt')['bnf_code'].astype('string').values 

baseline_gp_feids = set()
incident_gp_feids = set()

earliest_date_gp, n_baseline_invalid, n_incident_invalid = extract_gp_prescriptions(merged_gp_prescriptions_df, gp_bnf_med_codes, baseline_gp_feids, incident_gp_feids)
print(f'Invalid N for GP: baseline: {n_baseline_invalid} incident: {n_incident_invalid}')
print(f'Earliest GP diagnosis dates: \n{earliest_date_gp.describe()}')

# Set column values

baseline_med_from_gp = diagnosis_df['f.eid'].isin(baseline_gp_feids)
diagnosis_df.loc[baseline_med_from_gp, 'baseline_glaucoma_med'] = 1

incident_med_from_gp = diagnosis_df['f.eid'].isin(incident_gp_feids)
diagnosis_df.loc[incident_med_from_gp, 'incident_glaucoma_med'] = 1

Participants with invalid age for Dx in GP records: 0
Invalid N for GP: baseline: 0 incident: 0
Earliest GP diagnosis dates: 
count                              353
mean     2008-02-29 18:21:24.985835776
min                1989-04-21 00:00:00
25%                2003-11-21 00:00:00
50%                2009-02-23 00:00:00
75%                2013-05-07 00:00:00
max                2017-04-10 00:00:00
Name: issue_date, dtype: object


### Inpatient

In [ ]:
# Research group inpatient operation codes
OPCS3_codes = ["1512","1574"] #41273 OPCS3 surgical records
OPCS4_codes = ["C601","C602","C603","C604","C605","C606","C608","C609","C611","C612","C613","C614","C615","C618","C619","C651","C652","C653","C654","C655","C658","C659"] #41272 OPCS4 surgical records

In [34]:
### Operations

# f.eid lists for operations
baseline_inpatient_operation_feids = set()
incident_inpatient_operation_feids = set()

inpatient_operation_dates = pd.DataFrame(columns=['f.eid', 'earliest_inpatient_operation_date'])

# Cols
OPCS4_inpatient_operation_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41272.0')]
OPCS4_inpatient_operation_date_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41282.0')]

OPCS3_inpatient_operation_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41273.0')]
OPCS3_inpatient_operation_date_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41283.0')]


# OPCS4
process_inpatient_matrix(
    df = ukb49508_df,
    diagnosis_col_list = OPCS4_inpatient_operation_cols,
    date_col_list = OPCS4_inpatient_operation_date_cols,
    positive_values_list = OPCS4_codes,
    baseline_feid_set = baseline_inpatient_operation_feids,
    incident_feid_set = incident_inpatient_operation_feids,
    feid_dates_df = inpatient_operation_dates,
)

# OPCS3
process_inpatient_matrix(
    df = ukb49508_df,
    diagnosis_col_list = OPCS3_inpatient_operation_cols,
    date_col_list = OPCS3_inpatient_operation_date_cols,
    positive_values_list = OPCS3_codes,
    baseline_feid_set = baseline_inpatient_operation_feids,
    incident_feid_set = incident_inpatient_operation_feids,
    feid_dates_df = inpatient_operation_dates,
)

In [35]:
# Set column values

baseline_surgery_from_inpatient = diagnosis_df['f.eid'].isin(baseline_inpatient_operation_feids)
diagnosis_df.loc[baseline_surgery_from_inpatient, 'baseline_glaucoma_surgery'] = 1

incident_surgery_from_inpatient = diagnosis_df['f.eid'].isin(incident_inpatient_operation_feids)
diagnosis_df.loc[incident_surgery_from_inpatient, 'incident_glaucoma_surgery'] = 1

### Pool

In [36]:
baseline_glaucoma_surg = diagnosis_df['baseline_glaucoma_surgery'].eq(1)
incident_glaucoma_surg = diagnosis_df['incident_glaucoma_surgery'].eq(1)
invalid_glaucoma_surg = diagnosis_df['GP_glaucoma_surgery'] == 'Invalid'

diagnosis_df['prevalent_glaucoma_surgery'] = (baseline_glaucoma_surg | incident_glaucoma_surg | invalid_glaucoma_surg).astype(int)
diagnosis_df['prevalent_glaucoma_med'] = diagnosis_df[['baseline_glaucoma_med', 'incident_glaucoma_med']].eq(1).any(axis=1).astype(int)
diagnosis_df['prevalent_glaucoma_surgery_or_med'] = diagnosis_df[['prevalent_glaucoma_surgery', 'prevalent_glaucoma_med']].eq(1).any(axis=1).astype(int)

In [37]:
diagnosis_df['baseline_glaucoma_surgery_or_med'] = diagnosis_df[['baseline_glaucoma_surgery', 'baseline_glaucoma_med']].eq(1).any(axis=1).astype(int)
diagnosis_df['incident_glaucoma_surgery_or_med'] = diagnosis_df[['incident_glaucoma_surgery', 'incident_glaucoma_med']].eq(1).any(axis=1).astype(int)

# Glaucoma definitions

## Main outcome codes

In [39]:
# Glaucoma

f6148_codes = [2]
f20002_codes = [1277]
age_diagnosed_self_report_field = '4689'
read_v2_codes = ["F45..","F4501","F451.","F4510","F4511","F4512","F4513","F4515","F451z","F4540","F4551","F45y.","F45y2","F45yz","F45z.","FyuG0"]
read_v3_codes = ["F45..","F4501","F451.","F4510","F4511","F4512","F4513","F4515","F451z","F4540","F4551","F45y.","F45yz","F45z.","FyuG0","X00ee","XaF9D"]
inpatient_icd10_codes = ['H401', 'H408', 'H409']
inpatient_icd9_codes  = ['3651', '3658', '3659'] 

pooled_dx_col, pooled_age_col, sr_dx_col, sr_age_col, gp_dx_col, gp_date_col, gp_age_col, ip_dx_col, ip_date_col, ip_age_col = extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'Glaucoma',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
    skip_final_definitions = False,
)


Self-report
Baseline
Participants with invalid age in f4689: 157
Participants with disease in f6148, but NA age diagnosed in f4689: 4445
Incident
Participants with invalid age in f4689: 83
Baseline, participants with invalid age in f20009.0: 93
Incident, participants with invalid age in f20009.0: 35
Glaucoma_self_reported
Undiagnosed    493724
Baseline         7544
Incident         1151
Name: count, dtype: int64
count    6785.000000
mean       54.898394
std        11.031408
min         0.000000
25%        49.000000
50%        56.500000
75%        62.500000
max        80.400000
Name: Glaucoma_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 83
Glaucoma_GP_diagnosis
Undiagnosed    497129
Incident         2689
Baseline         2567
Invalid            34
Name: count, dtype: int64
count                             5256
mean     2007-07-18 18:03:33.698630400
min                1953-01-17 00:00:00
25%                2003-10-03 12:00:0

## Control exclusions

In [40]:
#### EXCLUSIONS
# Glaucoma, control exclusions

f6148_codes = None
f20002_codes = None
age_diagnosed_self_report_field = None
read_v2_codes = ["F442.","F4420","F4421","F4422","F4423","F442z","F45..","F450.","F4501","F4502","F4503","F450z","F451.","F4510","F4511","F4512","F4513","F4514","F4515","F451z","F452.","F4520","F4521","F4522","F4523","F4524","F452z","F453.","F4530","F4531","F453z","F454.","F4540","F4541","F4542","F4543","F4544","F454z","F455.","F4550","F4551","F455z","F456.","F4560","F4561","F4562","F4563","F4564","F4565","F4566","F456z","F45y.","F45y0","F45y1","F45y2","F45yz","F45z.","FyuG0","FyuG1","FyuG2","P32..","P320.","P3200","P3201","P320z","P321.","P322.","P3220","P3221","P322z","P32z.","P3420"]
read_v3_codes = ["F442.","F4420","F4421","F4422","F4423","F442z","F45..","F450.","F4501","F4502","F4503","F450z","F451.","F4510","F4511","F4512","F4513","F4514","F4515","F451z","F4520","F4521","F4522","F4523","F4524","F452z","F453.","F4530","F4531","F453z","F454.","F4540","F4541","F4542","F4543","F4544","F454z","F455.","F4550","F4551","F455z","F456.","F4560","F4561","F4562","F4563","F4564","F4565","F456z","F45y.","F45y0","F45y1","F45yz","F45z.","FyuG0","FyuG1","FyuG2","P32..","P320.","P3201","P320z","P321.","P322.","P3220","P322z","P32z.","P3420","X00ea","X00ed","X00ee","X00eg","X00eh","X00el","X00em","X00en","X00eo","X00eq","X00er","X00et","XaEVp","XaEVW","XaF9D","XE2a8"]
inpatient_icd10_codes = ["H400","H402","H403","H404","H405","H406","H420","H428","Q150"]
inpatient_icd9_codes = ["3650","3652","3655","3656"]

pooled_dx_col, pooled_age_col, sr_dx_col, sr_age_col, gp_dx_col, gp_date_col, gp_age_col, ip_dx_col, ip_date_col, ip_age_col = extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'glaucoma_control_exclusion',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
    skip_final_definitions = True,
)


Self-report
glaucoma_control_exclusion_self_reported
Undiagnosed    502419
Name: count, dtype: int64
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: glaucoma_control_exclusion_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 101
glaucoma_control_exclusion_GP_diagnosis
Undiagnosed    496265
Incident         3067
Baseline         3049
Invalid            38
Name: count, dtype: int64
count                          6116
mean     2007-03-22 10:47:57.174624
min             1949-01-01 00:00:00
25%             2003-03-20 00:00:00
50%             2009-04-04 12:00:00
75%             2012-12-12 00:00:00
max             2017-08-21 00:00:00
Name: glaucoma_control_exclusion_earliest_GP_date, dtype: object
count    6116.000000
mean       59.137345
std         9.648530
min         0.000000
25%        53.000000
50%        61.000000
75%        66.000000
max        79.000000
Name: glauc

## Final main definition

In [41]:
########## Definitions
### here: define control exclusions
### minor exclusions were defined above
####################

disease_name = 'Glaucoma'

# Exclusions for all
ever_pregnant = diagnosis_df['Ever pregnant'] == 1
discord_sex = diagnosis_df['Sex_nondisc'].isna()

# Exclusions for controls
incident_control_exclusion = diagnosis_df['glaucoma_control_exclusion_diagnosis_type'].eq('Incident')
incident_glacucoma_surgery_or_med = diagnosis_df['incident_glaucoma_surgery_or_med'].eq(1)

baseline_control_exclusion = diagnosis_df['glaucoma_control_exclusion_diagnosis_type'].eq('Baseline')
baseline_glacucoma_surgery_or_med = diagnosis_df['baseline_glaucoma_surgery_or_med'].eq(1)

# Invalid dates for GP or surgery exclusion codes ---> exclusions for controls
invalid_GP_exclusion = diagnosis_df['glaucoma_control_exclusion_GP_diagnosis'].eq('Invalid')
invalid_glaucoma_surgery = diagnosis_df['GP_glaucoma_surgery'].eq('Invalid')

# Pool control exclusions
control_exclusion = (diagnosis_df[f'Prevalent {disease_name} minor exclusions'] ==  0) & (
    incident_control_exclusion | incident_glacucoma_surgery_or_med
    | baseline_control_exclusion | baseline_glacucoma_surgery_or_med
    | diagnosis_df['prevalent_glaucoma_surgery_or_med']
    | invalid_GP_exclusion | invalid_glaucoma_surgery
)

### Prevalent definition control exclusions
prevalence_control_exclusion_col = f'Prevalent {disease_name} control exclusions'
diagnosis_df[prevalence_control_exclusion_col] = diagnosis_df[f'Prevalent {disease_name} minor exclusions']
diagnosis_df.loc[control_exclusion, prevalence_control_exclusion_col] = np.nan

print(f'\n{prevalence_control_exclusion_col}')
print(f'{diagnosis_df[prevalence_control_exclusion_col].value_counts()}')
print(f'NA: {diagnosis_df[prevalence_control_exclusion_col].isna().sum()}')


### Incident definition control exclusions
incident_control_exclusion_col = f'Incident {disease_name} control exclusions'
diagnosis_df[incident_control_exclusion_col] = diagnosis_df[f'Incident {disease_name} minor exclusions']
diagnosis_df.loc[control_exclusion, incident_control_exclusion_col] = np.nan

print(f'\n{incident_control_exclusion_col}')
print(f'{diagnosis_df[incident_control_exclusion_col].value_counts()}')
print(f'NA: {diagnosis_df[incident_control_exclusion_col].isna().sum()}')


### Baseline definition control exclusions
baseline_control_exclusion_col = f'Baseline {disease_name} control exclusions'
diagnosis_df[baseline_control_exclusion_col] = diagnosis_df[f'Baseline {disease_name} minor exclusions']
diagnosis_df.loc[control_exclusion, baseline_control_exclusion_col] = np.nan

print(f'\n{baseline_control_exclusion_col}')
print(f'{diagnosis_df[baseline_control_exclusion_col].value_counts()}')
print(f'NA: {diagnosis_df[baseline_control_exclusion_col].isna().sum()}')




Prevalent Glaucoma control exclusions
Prevalent Glaucoma control exclusions
0.0    481772
1.0     16154
Name: count, dtype: int64
NA: 4493

Incident Glaucoma control exclusions
Incident Glaucoma control exclusions
0.0    481772
1.0      7832
Name: count, dtype: int64
NA: 12815

Baseline Glaucoma control exclusions
Baseline Glaucoma control exclusions
0.0    481772
1.0      8311
Name: count, dtype: int64
NA: 12336


In [45]:
# N within IOP subcohort definition (properly defined later)

IOP_available_mask = ukb51745_df[['f.5262.0.0', 'f.5254.0.0']].isna().all(axis=1) == False
diagnosis_df.loc[IOP_available_mask, 'Prevalent Glaucoma control exclusions'].value_counts()

Prevalent Glaucoma control exclusions
0.0    108428
1.0      3728
Name: count, dtype: int64

## Alternative definition: Excluding diagnosed <40 years old

In [46]:
glaucoma_dx_below_40_or_na = (
    (diagnosis_df['Prevalent Glaucoma control exclusions'] == True) & (
        (diagnosis_df['Glaucoma_earliest_age_diagnosed'] <= 40)
        | (diagnosis_df['Glaucoma_earliest_age_diagnosed'].isna())
    )
)

print(glaucoma_dx_below_40_or_na.sum())

diagnosis_df['Prevalent Glaucoma control and age40 exclusions'] = diagnosis_df['Prevalent Glaucoma control exclusions']
diagnosis_df['Incident Glaucoma control and age40 exclusions'] = diagnosis_df['Incident Glaucoma control exclusions']
diagnosis_df['Baseline Glaucoma control and age40 exclusions'] = diagnosis_df['Baseline Glaucoma control exclusions']

diagnosis_df.loc[glaucoma_dx_below_40_or_na, 'Prevalent Glaucoma control and age40 exclusions'] = np.nan
diagnosis_df.loc[glaucoma_dx_below_40_or_na, 'Incident Glaucoma control and age40 exclusions'] = np.nan
diagnosis_df.loc[glaucoma_dx_below_40_or_na, 'Baseline Glaucoma control and age40 exclusions'] = np.nan

1674


# (TODO) Detailed surgeries for progression paper

## Identifying additional codes

In [ ]:
read_ctv3_opcs4_lkp = pd.read_excel('../data/coding/all_lkps_maps_v4.xlsx', sheet_name='read_ctv3_opcs4')
read_ctv3_lkp = pd.read_excel('../data/coding/all_lkps_maps_v4.xlsx', sheet_name='read_ctv3_lkp')

read_v2_opcs4_lkp = pd.read_excel('../data/coding/all_lkps_maps_v4.xlsx', sheet_name='read_v2_opcs4')
read_v2_lkp = pd.read_excel('../data/coding/all_lkps_maps_v4.xlsx', sheet_name='read_v2_lkp')

In [ ]:
opcs4_codes = [ 'C521', 'C522', 'C601', 'C605', 'C606', 'C612', 'C613', 'C618', 'C619']

In [ ]:
# CTv3 codes matching OPCS4

map_lkp = (
    read_ctv3_opcs4_lkp.loc[read_ctv3_opcs4_lkp["opcs4_code"].isin(opcs4_codes), ["read_code", "opcs4_code"]]
    .drop_duplicates()
)

# GP read codes that match those OPCS4 codes
matching_v3_gp_codes_opcs4 = map_lkp["read_code"].unique().tolist()

# Bring term descriptions + attach opcs4_code via merge
df = (
    read_ctv3_lkp.loc[read_ctv3_lkp["read_code"].isin(matching_v3_gp_codes_opcs4), ["read_code", "term_description"]]
    .drop_duplicates()
    .merge(map_lkp, on="read_code", how="left")
)

df

,read_code,term_description,opcs4_code
0,72550,Trabeculectomy,C601
1,72550,Creation of guarded fistula to sclera,C601
2,72550,Creation of subscleral fistula to sclera,C601
3,72554,Insertion anterior chamber drainage tube (& Mo...,C605
4,72554,Insertion of tube into anterior chamber of eye...,C605
5,72554,Insertion of Molteno implantation tube into an...,C605
6,7256.,Other operations on trabecular meshwork of eye,C619
7,72561,Trabeculotomy,C612
8,72562,Goniotomy (& Barkan),C613
9,72562,Goniotomy,C613


In [ ]:
# CTv2 codes matching OPCS4

map_lkp = (
    read_v2_opcs4_lkp.loc[read_v2_opcs4_lkp["opcs_4.2_code"].isin(opcs4_codes), ["read_code", "opcs_4.2_code"]]
    .drop_duplicates()
)

# GP read codes that match those OPCS4 codes
matching_v2_gp_codes_opcs4 = map_lkp["read_code"].unique().tolist()

# Bring term descriptions + attach opcs4_code via merge
df = (
    read_v2_lkp.loc[read_v2_lkp["read_code"].isin(matching_v2_gp_codes_opcs4), ["read_code", "term_description"]]
    .drop_duplicates()
    .merge(map_lkp, on="read_code", how="left")
)

df

,read_code,term_description,opcs_4.2_code
0,724A0,Deep sclerectomy with spacer,C521
1,724A1,Deep sclerectomy without spacer,C522
2,72550,Trabeculectomy,C601
3,72554,Insertion of tube into anterior chamber of eye...,C605
4,72554,Insertion of Molteno implantation tube into an...,C605
5,72555,Viscocanulostomy,C606
6,72557,Viscocanalostomy,C606
7,72561,Trabeculotomy,C612
8,72562,Goniotomy,C613
9,72562,Barkan goniotomy,C613


## GP code N
note - all prevalent counts
for QC

In [ ]:
read_v3_codes_df = pd.read_excel('./codes/progression_surgery_codes_outcomes.xlsx', sheet_name='read_v3')
read_v2_codes_df = pd.read_excel('./codes/progression_surgery_codes_outcomes.xlsx', sheet_name='read_v2')

read_v3_codes = read_v3_codes_df['Code'].astype(str).values.tolist()
read_v2_codes = read_v2_codes_df['Code'].astype(str).values.tolist()

In [ ]:
merged_gp_events_df['read_3'].isin(read_v3_codes).sum()

np.int64(595)

In [ ]:
counts = (
    merged_gp_events_df.loc[merged_gp_events_df["read_3"].isin(read_v3_codes)]
    .groupby("read_3")["f.eid"]
    .nunique()
    .reindex(read_v3_codes, fill_value=0)  
    .reset_index()
    .rename(columns={"read_3": "Code", "f.eid": "n_unique_f.eid"})
)

counts

,Code,n_unique_f.eid
0,XaL4t,2
1,724A0,0
2,724A1,0
3,XaL4u,2
4,X00Wp,1
5,XSAON,3
6,724A.,0
7,72521,0
8,72523,0
9,XaKaz,2


In [ ]:
merged_gp_events_df['read_2'].isin(read_v2_codes).sum()

np.int64(227)

In [ ]:
counts = (
    merged_gp_events_df.loc[merged_gp_events_df["read_2"].isin(read_v2_codes)]
    .groupby("read_2")["f.eid"]
    .nunique()
    .reindex(read_v2_codes, fill_value=0)  
    .reset_index()
    .rename(columns={"read_2": "Code", "f.eid": "n_unique_f.eid"})
)

counts

,read_2,n_unique_f.eid
0,724A0,0
1,724A1,0
2,72550,140
3,72554,2
4,72554,2
5,72557,1
6,72555,1
7,72561,5
8,72562,1
9,7256y,0


In [ ]:
read_v3_codes

['XaL4t',
 '724A0',
 '724A1',
 'XaL4u',
 'X00Wp',
 'XSAON',
 '724A.',
 '72521',
 '72523',
 'XaKaz',
 'XaKb1',
 'XaKb3',
 'X00X6',
 '72550',
 'XE0Km',
 '.7384',
 'XE0Ko',
 '.7385',
 'X00XI',
 '72551',
 '72554',
 '72554',
 '72554',
 'X00Wr',
 'XaJdX',
 'XE0BN',
 'X00Ws',
 'XaE6n',
 '72555',
 '72557',
 'XaE6n',
 '72561',
 '72562',
 'XE0BO',
 'XE0Kk',
 'Xa3mt',
 'Xa3mu',
 'Xa3mu',
 '.7381',
 '.7381',
 '.7381',
 '.7383',
 'X00X4',
 '7256z',
 '7256y',
 '7256.']

## Outcome criteria codes

### Self-reported

In [ ]:
# TODO: follow-up time

### Baseline

# Self-reported surgeries

# 5326 ever had surgery for glaucoma or OHT
positive_vals_5326 = [2, 3, 4]
baseline_surgery_from_5326 = ukb49508_df['f.5326.0.0'].isin(positive_vals_5326)

# 1436 (glaucoma surgery / trabeculectomy) in 20004
baseline_surgery_20004_feids = set()
cols_20004 = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.0')]
for col_idx in range(len(cols_20004)): 
    col = cols_20004[col_idx]
    surgery = ukb49508_df[col].eq(1436)
    baseline_surgery_20004_feids.update(ukb49508_df[surgery]['f.eid'])

baseline_surgery_from_20004 = ukb49508_df['f.eid'].isin(baseline_surgery_20004_feids)

### Incident

# Self-reported surgeries

# 5326 ever had surgery for glaucoma or OHT
cols_5326_inc = np.concatenate([
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5326.1')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5326.2')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.5326.3')],
])
positive_vals_5326 = [2, 3, 4]
incident_surgery_from_5326 = ukb49508_df[cols_5326_inc].isin(positive_vals_5326).any(axis=1)

# 1436 (glaucoma surgery / trabeculectomy) in 20004
incident_surgery_20004_feids = set()
cols_20004_inc = np.concatenate([
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.1')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.2')],
    ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.20004.3')],
])
for col_idx in range(len(cols_20004_inc)): 
    col = cols_20004_inc[col_idx]
    surgery = ukb49508_df[col].eq(1436)
    incident_surgery_20004_feids.update(ukb49508_df[surgery]['f.eid'])

incident_surgery_from_20004 = ukb49508_df['f.eid'].isin(incident_surgery_20004_feids)

## Summarise
baseline_mask = baseline_surgery_from_5326 | baseline_surgery_from_20004
incident_mask = incident_surgery_from_5326 | incident_surgery_from_20004

diagnosis_df['progression_outcome_glaucoma_surg_selfrep'] = 'None'
diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_surg_selfrep'] = 'Incident'
diagnosis_df.loc[baseline_mask, 'progression_outcome_glaucoma_surg_selfrep'] = 'Baseline' 

# TODO: interpoalted date
#diagnosis_df['progression_outcome_glaucoma_surg_selfrep_earliest_date'] = pd.NaT
#diagnosis_df.loc[baseline_mask | incident_mask, 'progression_outcome_glaucoma_surg_selfrep_earliest_date'] = diagnosis_df.loc[baseline_mask | incident_mask, 'f.eid'].map(earliest_date_gp)

print(f'Baseline {baseline_mask.sum()}')
print(f'Incident {incident_mask.sum()}')
print(f'Incident-only {(incident_mask & ~baseline_mask).sum()}')
print(diagnosis_df['progression_outcome_glaucoma_surg_selfrep'].value_counts())
#print(diagnosis_df['progression_outcome_glaucoma_surg_selfrep_earliest_date'].describe())

Baseline 831
Incident 142
Incident-only 112
progression_outcome_glaucoma_surg_selfrep
None        501476
Baseline       831
Incident       112
Name: count, dtype: int64


/tmp/ipykernel_3502080/3293839407.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_surg_selfrep'] = 'None'


### GP

In [ ]:
read_v3_codes_df = pd.read_excel('./codes/progression_surgery_codes_outcomes.xlsx', sheet_name='read_v3')
read_v2_codes_df = pd.read_excel('./codes/progression_surgery_codes_outcomes.xlsx', sheet_name='read_v2')


## Classification dummy vars
# read v3
cls = (
    read_v3_codes_df["Classification"]
    .fillna("")
    .astype(str)
    .str.split(",")
    .apply(lambda xs: {x.strip().upper() for x in xs if x.strip()})
)

read_v3_codes_df["I"] = cls.apply(lambda s: int("I" in s))
read_v3_codes_df["MIGS"] = cls.apply(lambda s: int("MIGS" in s))
read_v3_codes_df["D"] = cls.apply(lambda s: int("D" in s))
# v3
cls = (
    read_v2_codes_df["Classification"]
    .fillna("")
    .astype(str)
    .str.split(",")
    .apply(lambda xs: {x.strip().upper() for x in xs if x.strip()})
)

read_v2_codes_df["I"] = cls.apply(lambda s: int("I" in s))
read_v2_codes_df["MIGS"] = cls.apply(lambda s: int("MIGS" in s))
read_v2_codes_df["D"] = cls.apply(lambda s: int("D" in s))

## Lists
read_v3_codes = read_v3_codes_df['Code'].astype(str).values.tolist()
read_v2_codes = read_v2_codes_df['Code'].astype(str).values.tolist()

read_v3_codes_migs = read_v3_codes_df.loc[read_v3_codes_df["MIGS"] == 1, "Code"].astype(str).values.tolist()
read_v2_codes_migs = read_v2_codes_df.loc[read_v2_codes_df["MIGS"] == 1, "Code"].astype(str).values.tolist()

In [ ]:
# All surgeries

baseline_gp_feids = set()
incident_gp_feids = set()
invalid_gp_feids = set() # invalid should be an exclusion

earliest_date_gp, n_invalid = extract_gp_events(merged_gp_events_df, read_v2_codes, read_v3_codes, baseline_gp_feids, incident_gp_feids, invalid_gp_feids)
print(f'Earliest GP diagnosis dates: \n{earliest_date_gp.describe()}')

baseline_mask = diagnosis_df['f.eid'].isin(baseline_gp_feids)
incident_mask = diagnosis_df['f.eid'].isin(incident_gp_feids)
invalid_mask = diagnosis_df['f.eid'].isin(invalid_gp_feids)

diagnosis_df['progression_outcome_glaucoma_surg_gp'] = 'None'
diagnosis_df.loc[invalid_mask, 'progression_outcome_glaucoma_surg_gp'] = 'Invalid'  # set first. if has another date valid then is not invalid
diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_surg_gp'] = 'Incident'
diagnosis_df.loc[baseline_mask, 'progression_outcome_glaucoma_surg_gp'] = 'Baseline' 

diagnosis_df['progression_outcome_glaucoma_surg_gp_earliest_date'] = pd.NaT
diagnosis_df.loc[baseline_mask | incident_mask, 'progression_outcome_glaucoma_surg_gp_earliest_date'] = diagnosis_df.loc[baseline_mask | incident_mask, 'f.eid'].map(earliest_date_gp)

print(f'Baseline {baseline_mask.sum()}')
print(f'Incident {incident_mask.sum()}')
print(f'Incident-only {(incident_mask & ~baseline_mask).sum()}')
print(diagnosis_df['progression_outcome_glaucoma_surg_gp'].value_counts())
print(diagnosis_df['progression_outcome_glaucoma_surg_gp_earliest_date'].describe())

Participants with invalid age for Dx in GP records: 8
Earliest GP diagnosis dates: 
count                              490
mean     2006-07-21 11:39:25.714285824
min                1971-07-05 00:00:00
25%                2001-10-01 06:00:00
50%                2009-04-21 00:00:00
75%                2013-10-15 12:00:00
max                2017-04-12 00:00:00
Name: event_dt, dtype: object
Baseline 242
Incident 285
Incident-only 248
progression_outcome_glaucoma_surg_gp
None        501926
Incident       248
Baseline       242
Invalid          3
Name: count, dtype: int64
count                              490
mean     2006-07-21 11:39:25.714285824
min                1971-07-05 00:00:00
25%                2001-10-01 06:00:00
50%                2009-04-21 00:00:00
75%                2013-10-15 12:00:00
max                2017-04-12 00:00:00
Name: progression_outcome_glaucoma_surg_gp_earliest_date, dtype: object


/tmp/ipykernel_3502080/376039441.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_surg_gp'] = 'None'
/tmp/ipykernel_3502080/376039441.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_surg_gp_earliest_date'] = pd.NaT


In [ ]:
# MIGS specifically (for sensitivity analysis)

baseline_gp_feids = set()
incident_gp_feids = set()
invalid_gp_feids = set() # invalid should be an exclusion

earliest_date_gp, n_invalid = extract_gp_events(merged_gp_events_df, read_v2_codes_migs, read_v3_codes_migs, baseline_gp_feids, incident_gp_feids, invalid_gp_feids)
print(f'Earliest GP diagnosis dates: \n{earliest_date_gp.describe()}')

baseline_mask = diagnosis_df['f.eid'].isin(baseline_gp_feids)
incident_mask = diagnosis_df['f.eid'].isin(incident_gp_feids)
invalid_mask = diagnosis_df['f.eid'].isin(invalid_gp_feids)

diagnosis_df['progression_outcome_glaucoma_migs_gp'] = 'None'
diagnosis_df.loc[invalid_mask, 'progression_outcome_glaucoma_migs_gp'] = 'Invalid'  # set first. if has another date valid then is not invalid
diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_migs_gp'] = 'Incident'
diagnosis_df.loc[baseline_mask, 'progression_outcome_glaucoma_migs_gp'] = 'Baseline'   

diagnosis_df['progression_outcome_glaucoma_migs_gp_earliest_date'] = pd.NaT
diagnosis_df.loc[baseline_mask | incident_mask, 'progression_outcome_glaucoma_migs_gp_earliest_date'] = diagnosis_df.loc[baseline_mask | incident_mask, 'f.eid'].map(earliest_date_gp)


print(f'Baseline {baseline_mask.sum()}')
print(f'Incident {incident_mask.sum()}')
print(f'Incident-only {(incident_mask & ~baseline_mask).sum()}')
print(diagnosis_df['progression_outcome_glaucoma_migs_gp'].value_counts())
print(diagnosis_df['progression_outcome_glaucoma_migs_gp_earliest_date'].describe())

Participants with invalid age for Dx in GP records: 1
Earliest GP diagnosis dates: 
count                     40
mean     2009-01-08 02:24:00
min      1979-01-04 00:00:00
25%      2007-05-07 00:00:00
50%      2012-10-06 00:00:00
75%      2015-03-20 00:00:00
max      2016-04-08 00:00:00
Name: event_dt, dtype: object
Baseline 15
Incident 25
Incident-only 25
progression_outcome_glaucoma_migs_gp
None        502378
Incident        25
Baseline        15
Invalid          1
Name: count, dtype: int64
count                     40
mean     2009-01-08 02:24:00
min      1979-01-04 00:00:00
25%      2007-05-07 00:00:00
50%      2012-10-06 00:00:00
75%      2015-03-20 00:00:00
max      2016-04-08 00:00:00
Name: progression_outcome_glaucoma_migs_gp_earliest_date, dtype: object


/tmp/ipykernel_3502080/2567904380.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_migs_gp'] = 'None'
/tmp/ipykernel_3502080/2567904380.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_migs_gp_earliest_date'] = pd.NaT


### Inpatient

In [ ]:
opcs4_codes_df = pd.read_excel('./codes/progression_surgery_codes_outcomes.xlsx', sheet_name='opcs4_inpatient')
opcs3_codes_df = pd.read_excel('./codes/progression_surgery_codes_outcomes.xlsx', sheet_name='opcs3_inpatient')

## Classification dummy vars
# OPCS4
cls = (
    opcs4_codes_df["Classification"]
    .fillna("")
    .astype(str)
    .str.split(",")
    .apply(lambda xs: {x.strip().upper() for x in xs if x.strip()})
)

opcs4_codes_df["I"] = cls.apply(lambda s: int("I" in s))
opcs4_codes_df["MIGS"] = cls.apply(lambda s: int("MIGS" in s))
opcs4_codes_df["D"] = cls.apply(lambda s: int("D" in s))

# OPCS3
cls = (
    opcs3_codes_df["Classification"]
    .fillna("")
    .astype(str)
    .str.split(",")
    .apply(lambda xs: {x.strip().upper() for x in xs if x.strip()})
)

opcs3_codes_df["I"] = cls.apply(lambda s: int("I" in s))
opcs3_codes_df["MIGS"] = cls.apply(lambda s: int("MIGS" in s))
opcs3_codes_df["D"] = cls.apply(lambda s: int("D" in s))

## Lists
opcs4_codes = opcs4_codes_df['Code'].astype(str).values.tolist()
opcs3_codes = opcs3_codes_df['Code'].astype(str).values.tolist()

opcs4_codes_migs = opcs4_codes_df.loc[opcs4_codes_df["MIGS"] == 1, "Code"].astype(str).values.tolist()
opcs3_codes_migs = opcs3_codes_df.loc[opcs3_codes_df["MIGS"] == 1, "Code"].astype(str).values.tolist()

In [ ]:
# Cols
OPCS4_inpatient_operation_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41272.0')]
OPCS4_inpatient_operation_date_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41282.0')]

OPCS3_inpatient_operation_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41273.0')]
OPCS3_inpatient_operation_date_cols = ukb49508_df.columns[ukb49508_df.columns.str.startswith('f.41283.0')]

In [ ]:
### All operations

baseline_inpatient_operation_feids = set()
incident_inpatient_operation_feids = set()
inpatient_operation_dates = pd.DataFrame(columns=['f.eid', 'earliest_inpatient_operation_date'])

# OPCS4
process_inpatient_matrix(
    df = ukb49508_df,
    diagnosis_col_list = OPCS4_inpatient_operation_cols,
    date_col_list = OPCS4_inpatient_operation_date_cols,
    positive_values_list = OPCS4_codes,
    baseline_feid_set = baseline_inpatient_operation_feids,
    incident_feid_set = incident_inpatient_operation_feids,
    feid_dates_df = inpatient_operation_dates,
)

# OPCS3
process_inpatient_matrix(
    df = ukb49508_df,
    diagnosis_col_list = OPCS3_inpatient_operation_cols,
    date_col_list = OPCS3_inpatient_operation_date_cols,
    positive_values_list = OPCS3_codes,
    baseline_feid_set = baseline_inpatient_operation_feids,
    incident_feid_set = incident_inpatient_operation_feids,
    feid_dates_df = inpatient_operation_dates,
)

# Set columns
baseline_mask = diagnosis_df['f.eid'].isin(baseline_inpatient_operation_feids)
incident_mask = diagnosis_df['f.eid'].isin(incident_inpatient_operation_feids)

diagnosis_df['progression_outcome_glaucoma_surg_ip'] = 'None'
diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_surg_ip'] = 'Incident'
diagnosis_df.loc[baseline_mask, 'progression_outcome_glaucoma_surg_ip'] = 'Baseline'   

diagnosis_df['progression_outcome_glaucoma_surg_ip_earliest_date'] = pd.NaT
diagnosis_df.loc[baseline_mask | incident_mask, 'progression_outcome_glaucoma_surg_ip_earliest_date'] = diagnosis_df.loc[baseline_mask | incident_mask, 'f.eid'].map(inpatient_operation_dates.groupby('f.eid')['earliest_inpatient_operation_date'].min())


print(f'Baseline {baseline_mask.sum()}')
print(f'Incident {incident_mask.sum()}')
print(f'Incident-only {(incident_mask & ~baseline_mask).sum()}')
print(diagnosis_df['progression_outcome_glaucoma_surg_ip'].value_counts())
print(diagnosis_df['progression_outcome_glaucoma_surg_ip_earliest_date'].describe())

Baseline 533
Incident 2540
Incident-only 2433
progression_outcome_glaucoma_surg_ip
None        499453
Incident      2433
Baseline       533
Name: count, dtype: int64
count                             2966
mean     2014-01-30 10:54:27.430883328
min                1989-08-23 00:00:00
25%                2011-01-21 18:00:00
50%                2015-11-16 00:00:00
75%                2018-09-10 00:00:00
max                2021-03-31 00:00:00
Name: progression_outcome_glaucoma_surg_ip_earliest_date, dtype: object


/tmp/ipykernel_3502080/2475909253.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_surg_ip'] = 'None'
/tmp/ipykernel_3502080/2475909253.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_surg_ip_earliest_date'] = pd.NaT


In [ ]:
# MIGS only

baseline_inpatient_operation_feids = set()
incident_inpatient_operation_feids = set()
inpatient_operation_dates = pd.DataFrame(columns=['f.eid', 'earliest_inpatient_operation_date'])

# OPCS4
process_inpatient_matrix(
    df = ukb49508_df,
    diagnosis_col_list = OPCS4_inpatient_operation_cols,
    date_col_list = OPCS4_inpatient_operation_date_cols,
    positive_values_list = opcs4_codes_migs,
    baseline_feid_set = baseline_inpatient_operation_feids,
    incident_feid_set = incident_inpatient_operation_feids,
    feid_dates_df = inpatient_operation_dates,
)

# OPCS3
process_inpatient_matrix(
    df = ukb49508_df,
    diagnosis_col_list = OPCS3_inpatient_operation_cols,
    date_col_list = OPCS3_inpatient_operation_date_cols,
    positive_values_list = opcs3_codes_migs,
    baseline_feid_set = baseline_inpatient_operation_feids,
    incident_feid_set = incident_inpatient_operation_feids,
    feid_dates_df = inpatient_operation_dates,
)

# Set columns
baseline_mask = diagnosis_df['f.eid'].isin(baseline_inpatient_operation_feids)
incident_mask = diagnosis_df['f.eid'].isin(incident_inpatient_operation_feids)

diagnosis_df['progression_outcome_glaucoma_migs_ip'] = 'None'
diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_migs_ip'] = 'Incident'
diagnosis_df.loc[baseline_mask, 'progression_outcome_glaucoma_migs_ip'] = 'Baseline'   

diagnosis_df['progression_outcome_glaucoma_migs_ip_earliest_date'] = pd.NaT
diagnosis_df.loc[baseline_mask | incident_mask, 'progression_outcome_glaucoma_migs_ip_earliest_date'] = diagnosis_df.loc[baseline_mask | incident_mask, 'f.eid'].map(inpatient_operation_dates.groupby('f.eid')['earliest_inpatient_operation_date'].min())


print(f'Baseline {baseline_mask.sum()}')
print(f'Incident {incident_mask.sum()}')
print(f'Incident-only {(incident_mask & ~baseline_mask).sum()}')
print(diagnosis_df['progression_outcome_glaucoma_migs_ip'].value_counts())
print(diagnosis_df['progression_outcome_glaucoma_migs_ip_earliest_date'].describe())

Baseline 84
Incident 733
Incident-only 730
progression_outcome_glaucoma_migs_ip
None        501605
Incident       730
Baseline        84
Name: count, dtype: int64
count                              814
mean     2015-12-02 10:58:05.012285440
min                1990-06-25 00:00:00
25%                2014-08-03 00:00:00
50%                2017-10-09 12:00:00
75%                2019-07-20 18:00:00
max                2021-03-30 00:00:00
Name: progression_outcome_glaucoma_migs_ip_earliest_date, dtype: object


/tmp/ipykernel_3502080/1585866634.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_migs_ip'] = 'None'
/tmp/ipykernel_3502080/1585866634.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['progression_outcome_glaucoma_migs_ip_earliest_date'] = pd.NaT


### Pooled columns

In [ ]:
# only defined using inpatient or GP
# NOT self report

any_baseline_surg = diagnosis_df['progression_outcome_glaucoma_surg_gp'].eq('Baseline') | diagnosis_df['progression_outcome_glaucoma_surg_ip'].eq('Baseline')
any_incident_surg = diagnosis_df['progression_outcome_glaucoma_surg_gp'].eq('Incident') | diagnosis_df['progression_outcome_glaucoma_surg_ip'].eq('Incident')
baseline_mask = any_baseline_surg
incident_mask = any_incident_surg & ~baseline_mask

diagnosis_df['progression_outcome_glaucoma_surg_pooled'] = 'None'
diagnosis_df.loc[baseline_mask, 'progression_outcome_glaucoma_surg_pooled'] = 'Baseline'
diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_surg_pooled'] = 'Incident'

# Earliest date of surgery
diagnosis_df['progression_outcome_glaucoma_surg_pooled_earliest_date'] = diagnosis_df[['progression_outcome_glaucoma_surg_gp_earliest_date', 'progression_outcome_glaucoma_surg_ip_earliest_date']].min(axis=1)

# Time to event
diagnosis_df['progression_outcome_glaucoma_surg_pooled_time_to_days'] = (diagnosis_df['progression_outcome_glaucoma_surg_pooled_earliest_date'] - ukb49508_df['f.53.0.0']).dt.days

print(any_baseline_surg.sum())
print(any_incident_surg.sum())
print(incident_mask.sum())
print(diagnosis_df['progression_outcome_glaucoma_surg_pooled'].value_counts())
print(diagnosis_df['progression_outcome_glaucoma_surg_pooled_earliest_date'].describe())
print(diagnosis_df['progression_outcome_glaucoma_surg_pooled_time_to_days'].describe())

print('\nDates among incident cases')
print(diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_surg_pooled_earliest_date'].describe())
print(diagnosis_df.loc[incident_mask, 'progression_outcome_glaucoma_surg_pooled_time_to_days'].describe())

643
2488
2459
progression_outcome_glaucoma_surg_pooled
None        499317
Incident      2459
Baseline       643
Name: count, dtype: int64
count                             3102
mean     2013-05-16 14:59:39.110251520
min                1971-07-05 00:00:00
25%                2010-06-23 06:00:00
50%                2015-07-24 12:00:00
75%                2018-07-18 00:00:00
max                2021-03-31 00:00:00
Name: progression_outcome_glaucoma_surg_pooled_earliest_date, dtype: object
count     3102.000000
mean      1579.682463
std       2579.404268
min     -14204.000000
25%        540.250000
50%       2380.500000
75%       3423.750000
max       5390.000000
Name: progression_outcome_glaucoma_surg_pooled_time_to_days, dtype: float64

Dates among incident cases
count                             2459
mean     2016-05-11 21:37:06.758845184
min                2007-10-22 00:00:00
25%                2013-12-14 12:00:00
50%                2016-12-13 00:00:00
75%                2019-01-17 00:00:00

In [ ]:
# todo: exclusions
#### exclude invalid GP date (treat as baseline)

## Exclusion criteria codes

## Defining overall cohort (exclusions and outcomes)

# Cataract

In [47]:
# Cataract

f6148_codes = [4]
f20002_codes = [1278]
age_diagnosed_self_report_field = '4700'
read_v2_codes = ["F462.","F4620","F4621","F4622","F4623","F462z","F464.","F4643","F4644","F4645","F4646","F4647","F464z","F465.","F4650","F4651","F4652","F4653","F4655","F465z","F466.","F46y.","F46y0","F46yz","F46z.","F46..","F460.","F4600","F4601","F4602","F4603","F4604","F4605","F4606","F4607","F460x","F460y","F460z","F4610","F4611","F4613","F4614","F4615","F4616","F4617","F4618","F4619","F461A","F461B","F461x","F461y","F461z","F462.","F463.","F4630","F4631","F4632","F4633","F4634","F463z","F4641","F46z0","F4K34","FyuE0","FyuE1"]
read_v3_codes = ["22E5.","F46..","F4605","F4607","F462.","F4620","F4622","F4623","F462z","F464.","F4643","F4644","F4645","F4646","F4647","F464z","F465.","F4650","F4651","F4652","F4653","F465z","F46y.","F46y0","F46yz","F46z.","X00c7","X00cA","X00cB","X00cC","X00cE","X00cG","X00cH","X00cI","X00cJ","X77sb","XE2QA","XM00J","Xa0lP","22EE.","F460.","F4600","F4601","F4602","F4603","F4604","F4606","F460x","F460y","F460z","F461.","F4610","F4611","F4613","F4614","F4615","F4616","F4617","F4618","F4619","F461A","F461B","F461x","F461y","F461z","F463.","F4630","F4631","F4632","F4633","F4634","F463z","F4641","F466.","F4K34","FyuE0","FyuE1","P3311","X75kX","X75kb","X75kc","X75kd","X75kh","X75ki","X75kj","X75kk","X75kl","X75km","X75kn","X75ko","X75kp","X75l1","XE16G","XaBLO","XaBLP","XaD2a","XaEVU","XaFTm","XaG28","XaG29"]
inpatient_icd10_codes = ["H250","H251","H252","H258","H259","H260","H261","H262","H263","H264","H268","H269","H280","H281","H282","H288"]
inpatient_icd9_codes  = ["3662","3664","3665","3668","3669"]

pooled_dx_col, pooled_age_col, sr_dx_col, sr_age_col, gp_dx_col, gp_date_col, gp_age_col, ip_dx_col, ip_date_col, ip_age_col = extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'Cataract',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
    skip_final_definitions = True,
)


Self-report
Baseline
Participants with invalid age in f4700: 213
Participants with disease in f6148, but NA age diagnosed in f4700: 8324
Incident
Participants with invalid age in f4700: 336
Baseline, participants with invalid age in f20009.0: 105
Incident, participants with invalid age in f20009.0: 57
Cataract_self_reported
Undiagnosed    478067
Baseline        16998
Incident         7354
Name: count, dtype: int64
count    17466.000000
mean        60.429932
std         11.729918
min          0.000000
25%         57.000000
50%         62.950000
75%         67.000000
max         82.100000
Name: Cataract_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 40
Cataract_GP_diagnosis
Undiagnosed    493099
Incident         5818
Baseline         3482
Invalid            20
Name: count, dtype: int64
count                             9300
mean     2009-07-19 03:20:49.548387072
min                1950-01-01 00:00:00
25%                2006-09-

In [48]:
# Self-reported surgery

positive_vals = [2, 3, 4]
baseline_cataract_surgery = ukb49508_df['f.5324.0.0'].isin(positive_vals)
incident_cataract_surgery = ukb49508_df[['f.5324.1.0']].isin(positive_vals).any(axis=1)

diagnosis_df['baseline_cataract_surgery'] = baseline_cataract_surgery.astype(int)
diagnosis_df['incident_cataract_surgery'] = incident_cataract_surgery.astype(int)
diagnosis_df['prevalent_cataract_surgery'] = diagnosis_df[['baseline_cataract_surgery', 'incident_cataract_surgery']].any(axis=1).astype(int)

In [49]:
diagnosis_df.loc[diagnosis_df['IOP_subcohort'] == 1, 'Glaucoma_diagnosis_type'].value_counts()

Glaucoma_diagnosis_type
Control     108461
Baseline      2143
Incident      1549
Name: count, dtype: int64

In [50]:
#### Final definitions, incorporating cataract surgery too
###################################

# using code from func
output_df = diagnosis_df
col_name = 'Cataract'

# Diagnosis
incident_self_report = output_df[sr_dx_col].eq('Incident')
incident_GP          = output_df[gp_dx_col].eq('Incident')
incident_inpatient   = output_df[ip_dx_col].eq('Incident')
any_incident_dx      = incident_self_report | incident_GP | incident_inpatient | incident_cataract_surgery

baseline_self_report = output_df[sr_dx_col].eq('Baseline')
baseline_GP          = output_df[gp_dx_col].eq('Baseline')
baseline_inpatient   = output_df[ip_dx_col].eq('Baseline')
any_baseline_dx      = baseline_self_report | baseline_GP | baseline_inpatient | baseline_cataract_surgery

# Baseline overrides incident if both
any_dx = any_baseline_dx | any_incident_dx
baseline_mask = any_baseline_dx
incident_mask = any_incident_dx & ~baseline_mask

labels = np.full(len(output_df), 'Control', dtype=object)
labels[incident_mask] = 'Incident'
labels[baseline_mask] = 'Baseline'

pooled_dx_col = f'{col_name}_diagnosis_type'
output_df[pooled_dx_col] = labels

# Earliest age column
## (Does not include surgically defined individuals, if any)
pooled_age_col = f'{col_name}_earliest_age_diagnosed'
output_df[pooled_age_col] = output_df[[sr_age_col, gp_age_col, ip_age_col]].min(axis=1, skipna=True)

print(output_df[pooled_dx_col].value_counts())
print(output_df[pooled_age_col].describe())

#### Definitions with exclusions
###################################
# Exclusions for all
ever_pregnant = output_df['Ever pregnant'] == 1
discord_sex = output_df['Sex_nondisc'].isna()

# invalid GP date with no other dx source
# Include as prevalent case, but exclude from study baseline/incident
invalid_gp_mask = output_df[gp_dx_col] == 'Invalid'
invalid_gp_no_other_dx = invalid_gp_mask & (output_df[pooled_dx_col] == 'Control')

#### Prevalence
prevalence_col = f'Prevalent {col_name} minor exclusions'
output_df[prevalence_col] = output_df[pooled_dx_col].isin(['Baseline', 'Incident']).astype(int)
output_df.loc[invalid_gp_no_other_dx, prevalence_col] = 1 # GP invalid date issue
output_df.loc[ever_pregnant | discord_sex, prevalence_col] = np.nan # minor exclusions
print(f'\n{prevalence_col}')
print(f'{output_df[prevalence_col].value_counts()}')
print(f'NA: {output_df[prevalence_col].isna().sum()}')

### Incident
incident_col = f'Incident {col_name} minor exclusions'
output_df[incident_col] = output_df[pooled_dx_col].eq('Incident').astype(int)
output_df.loc[invalid_gp_no_other_dx, incident_col] = np.nan # GP invalid date issue
output_df.loc[ever_pregnant | discord_sex, incident_col] = np.nan # minor exclusions
output_df.loc[output_df[pooled_dx_col] == 'Baseline', incident_col] = np.nan # exclude baseline dx
print(f'\n{incident_col}')
print(f'{output_df[incident_col].value_counts()}')
print(f'NA: {output_df[incident_col].isna().sum()}')


### Baseline
baseline_col = f'Baseline {col_name} minor exclusions'
output_df[baseline_col] = output_df[pooled_dx_col].eq('Baseline').astype(int)
output_df.loc[invalid_gp_no_other_dx, baseline_col] = np.nan # GP invalid date issue
output_df.loc[ever_pregnant | discord_sex, baseline_col] = np.nan # minor exclusions
output_df.loc[output_df[pooled_dx_col] == 'Incident', baseline_col] = np.nan # exclude incident dx
print(f'\n{baseline_col}')
print(f'{output_df[baseline_col].value_counts()}')
print(f'NA: {output_df[baseline_col].isna().sum()}')

Cataract_diagnosis_type
Control     433106
Incident     47820
Baseline     21493
Name: count, dtype: int64
count    66812.000000
mean        66.113709
std          9.647793
min          0.000000
25%         61.800000
50%         68.000000
75%         73.000000
max         85.000000
Name: Cataract_earliest_age_diagnosed, dtype: float64

Prevalent Cataract minor exclusions
Prevalent Cataract minor exclusions
0.0    432660
1.0     69234
Name: count, dtype: int64
NA: 525

Incident Cataract minor exclusions
Incident Cataract minor exclusions
0.0    432660
1.0     47772
Name: count, dtype: int64
NA: 21987

Baseline Cataract minor exclusions
Baseline Cataract minor exclusions
0.0    432660
1.0     21458
Name: count, dtype: int64
NA: 48301


# AMD

In [51]:
# AMD

f6148_codes = [5]
f20002_codes = [1528]
age_diagnosed_self_report_field = '5923'
read_v2_codes = ["F425.","F4250","F4251","F4252","F4253","F4254","F4255","F4256","F4257","F425z","F4258","F4259","F427G","F4343"]
read_v3_codes = ["F4251","F4252","F4253","F4256","F4257","F425z","F427G","X002t","X00cx","X00d1","X00dX","X00eJ","X00eK","X75nI","X75nS","XE15x","XE15y","Xa9BN","F425.","F4250","F4254","F4255","F4343","X00eF","X00eL","X75mb","X75mc","X75md","X75mg","X75mk","X75mp","X75mr","X75n2","X75o7","XE18j","Xa9BO","XaE0Y","XaE0Z","XaE0a","XaE0b","XaE5J","XaE5R","XaE5d","XaE5t","XaF41"]
inpatient_icd10_codes = ['H353']
inpatient_icd9_codes  = ['3625']

extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'AMD',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
)


Self-report
Baseline
Participants with invalid age in f5923: 70
Participants with disease in f6148, but NA age diagnosed in f5923: 2567
Incident
Participants with invalid age in f5923: 92
Baseline, participants with invalid age in f20009.0: 0
Incident, participants with invalid age in f20009.0: 1
AMD_self_reported
Undiagnosed    497262
Baseline         3917
Incident         1240
Name: count, dtype: int64
count    2653.000000
mean       59.828345
std        10.929982
min         0.000000
25%        55.000000
50%        62.000000
75%        66.700000
max        81.200000
Name: AMD_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 19
AMD_GP_diagnosis
Undiagnosed    499149
Incident         2105
Baseline         1157
Invalid             8
Name: count, dtype: int64
count                             3262
mean     2009-12-13 08:15:44.696505088
min                1961-03-07 00:00:00
25%                2007-04-05 00:00:00
50%             

('AMD_diagnosis_type',
 'AMD_earliest_age_diagnosed',
 'AMD_self_reported',
 'AMD_earliest_age_diagnosed_self_report',
 'AMD_GP_diagnosis',
 'AMD_earliest_GP_date',
 'AMD_earliest_age_diagnosed_GP',
 'AMD_inpatient_diagnosis',
 'AMD_earliest_inpatient_date',
 'AMD_earliest_age_diagnosed_inpatient')

# Diabetes

In [52]:
# T1DM and T2DM

f6148_codes = None
f20002_codes = [1222, 1223]
age_diagnosed_self_report_field = None
read_v2_codes = ["C10..","C1000","C1010","C1020","C1030","C1040","C1050","C1060","C1070","C1073","C108.","C1080","C1081","C1082","C1083","C1084","C1085","C1086","C1087","C1088","C1089","C108A","C108B","C108C","C108D","C108E","C108F","C108G","C108H","C108J","C10E.","C10E0","C10E1","C10E2","C10E3","C10E4","C10E5","C10E6","C10E7","C10E8","C10E9","C10EA","C10EB","C10EC","C10ED","C10EE","C10EF","C10EG","C10EH","C10EJ","C10EK","C10EL","C10EM","C10EN","C10EP","C10EQ","C10ER","C10P.","C10P0","C10y0","C10z0","Cyu2.", "C1001","C1011","C1021","C1031","C1041","C1051","C1061","C1071","C1072","C1074","C109.","C1090","C1091","C1092","C1093","C1094","C1095","C1096","C1097","C1099","C109A","C109B","C109C","C109D","C109E","C109F","C109G","C109H","C109J","C109K","C10F.","C10F0","C10F1","C10F2","C10F3","C10F4","C10F5","C10F6","C10F7","C10F8","C10F9","C10FA","C10FB","C10FC","C10FD","C10FE","C10FF","C10FG","C10FH","C10FJ","C10FK","C10FL","C10FM","C10FN","C10FP","C10FQ","C10FR","C10FS","C10P1","C10Q.","C10y1","C10z1"] 
read_v3_codes = ["C10..","C10..","C1010","C1010","C1010","C1020","C1030","C1030","C1030","C1040","C1050","C1060","C1070","C1073","C1080","C1080","C1080","C1080","C1080","C1080","C1081","C1081","C1081","C1081","C1081","C1081","C1082","C1082","C1082","C1082","C1082","C1082","C1083","C1083","C1083","C1083","C1083","C1083","C1083","C1083","C1085","C1085","C1085","C1085","C1085","C1085","C1085","C1085","C1086","C1086","C1086","C1086","C1086","C1086","C1086","C1086","C1087","C1087","C1087","C1087","C1087","C1087","C1087","C1087","C1088","C1088","C1088","C1088","C1088","C1088","C1088","C1088","C1089","C1089","C1089","C1089","C1089","C1089","C1089","C1089","C10y0","C10z0","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","X40J4","Xa4g7","Xa4g7","Xa4g7","Xa4g7","Xa4g7","Xa4g7","Xaagd","Xaage","Xaage","XaELP","XaELP","XaELP","XaELP","XaELP","XaELP","XaEnn","XaEnn","XaEnn","XaEnn","XaEnn","XaEnn","XaEno","XaEno","XaEno","XaEno","XaEno","XaEno","XaF04","XaF04","XaF04","XaF04","XaF04","XaF04","XaFm8","XaFm8","XaFm8","XaFm8","XaFm8","XaFm8","XaFmK","XaFmK","XaFmK","XaFmK","XaFmK","XaFmK","XaFmL","XaFmL","XaFmL","XaFmL","XaFmL","XaFmL","XaFmM","XaFmM","XaFmM","XaFmM","XaFmM","XaFmM","XaFWG","XaFWG","XaFWG","XaFWG","XaFWG","XaFWG","XaIzM","XaIzM","XaIzN","XaIzN","XaJSr","XaJSr","XaKyW","XaKyW","XaOPu","XE10E", "C1011","C1011","C1011","C1021","C1031","C1031","C1031","C1031","C1041","C1051","C1061","C1071","C1072","C1074","C1090","C1090","C1090","C1090","C1090","C1091","C1091","C1091","C1091","C1091","C1092","C1092","C1092","C1092","C1092","C1093","C1093","C1093","C1093","C1093","C1094","C1094","C1094","C1094","C1094","C1094","C1095","C1095","C1095","C1095","C1095","C1095","C1096","C1096","C1096","C1096","C1096","C1097","C1097","C1097","C1097","C1097","C1097","C10y1","C10z1","X40J5","X40J5","X40J5","X40J5","X40J5","X40J5","X40J5","X40J5","X40J5","X40J5","X40J5","X40J6","X40J6","X40J6","X40J6","X40J6","X40JE","X40JE","Xaagf","Xaagf","XacoB","XaELQ","XaELQ","XaELQ","XaELQ","XaELQ","XaEnp","XaEnp","XaEnp","XaEnp","XaEnp","XaEnq","XaEnq","XaEnq","XaEnq","XaEnq","XaF05","XaF05","XaF05","XaF05","XaF05","XaFmA","XaFmA","XaFmA","XaFmA","XaFmA","XaFn7","XaFn7","XaFn7","XaFn7","XaFn7","XaFn8","XaFn8","XaFn8","XaFn8","XaFn8","XaFn9","XaFn9","XaFn9","XaFn9","XaFn9","XaFWI","XaFWI","XaFWI","XaFWI","XaFWI","XaIrf","XaIrf","XaIrf","XaIzQ","XaIzQ","XaIzQ","XaIzR","XaIzR","XaIzR","XaJQp","XaJQp","XaJQp","XaKyX","XaKyX","XaOPt","XE10F"] 
inpatient_icd10_codes = ["E10","E100","E101","E102","E103","E104","E105","E106","E107","E108","E109","E11","E110","E111","E112","E113","E114","E115","E116","E117","E118","E119"]
inpatient_icd9_codes = []

extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'T1orT2DM',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
)


Self-report
Baseline, participants with invalid age in f20009.0: 16
Incident, participants with invalid age in f20009.0: 4
T1orT2DM_self_reported
Undiagnosed    497455
Baseline         3803
Incident         1161
Name: count, dtype: int64
count    4944.000000
mean       52.518871
std        12.051844
min         0.500000
25%        47.500000
50%        55.500000
75%        60.500000
max        80.100000
Name: T1orT2DM_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 317
T1orT2DM_GP_diagnosis
Undiagnosed    483301
Baseline        10446
Incident         8650
Invalid            22
Name: count, dtype: int64
count                         19096
mean     2006-09-12 22:05:18.223712
min             1944-01-01 00:00:00
25%             2003-01-15 00:00:00
50%             2008-04-10 00:00:00
75%             2012-09-05 06:00:00
max             2017-09-15 00:00:00
Name: T1orT2DM_earliest_GP_date, dtype: object
count    19096.000000
mean      

('T1orT2DM_diagnosis_type',
 'T1orT2DM_earliest_age_diagnosed',
 'T1orT2DM_self_reported',
 'T1orT2DM_earliest_age_diagnosed_self_report',
 'T1orT2DM_GP_diagnosis',
 'T1orT2DM_earliest_GP_date',
 'T1orT2DM_earliest_age_diagnosed_GP',
 'T1orT2DM_inpatient_diagnosis',
 'T1orT2DM_earliest_inpatient_date',
 'T1orT2DM_earliest_age_diagnosed_inpatient')

# DR 

In [53]:
# DR

f6148_codes = [1]
f20002_codes = [1276]
age_diagnosed_self_report_field = '5901'
read_v2_codes = ["F420.","F4200","F4201","F4202","F4203","F420z","F420.","F4204","F4205","F4206","F4207","F4208"]
read_v3_codes = ["C1087","C1096","F420.","F4200","F4201","F4202","F4203","F420z","X00dF","X00dG","X00dH","X00dI","X00dJ","XaBul","XaD2T","XaE5T","XaE5U","XaE5V","XaE5W","XaE5X","XaE5Y","XaE5Z","XaE5c","XaEVO","XaEVP","XaEVS","XaEVT","XaIP5","XaIPk","XaIW8","XaJOg","XaJOh","XaJOi","XaJOj","XaJOk","XaJOl","XaJOn","XaJOo","XaJQp","XaJSr","XaKDG","XaKDH","XaKDI","XaKDJ","XaKcS","XaPen","XaXfs"]
inpatient_icd10_codes = ["H360"]
inpatient_icd9_codes  = ["3620"]

extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'DR',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
)


Self-report
Baseline
Participants with invalid age in f5901: 149
Participants with disease in f6148, but NA age diagnosed in f5901: 1872
Incident
Participants with invalid age in f5901: 48
Baseline, participants with invalid age in f20009.0: 59
Incident, participants with invalid age in f20009.0: 11
DR_self_reported
Undiagnosed    498215
Baseline         3725
Incident          479
Name: count, dtype: int64
count    2302.000000
mean       52.998871
std        13.035508
min         0.000000
25%        48.000000
50%        55.500000
75%        61.000000
max        81.200000
Name: DR_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 24
DR_GP_diagnosis
Undiagnosed    496069
Incident         3917
Baseline         2429
Invalid             4
Name: count, dtype: int64
count                             6346
mean     2009-09-08 06:38:00.491648512
min                1966-10-25 00:00:00
25%                2007-07-26 00:00:00
50%             

('DR_diagnosis_type',
 'DR_earliest_age_diagnosed',
 'DR_self_reported',
 'DR_earliest_age_diagnosed_self_report',
 'DR_GP_diagnosis',
 'DR_earliest_GP_date',
 'DR_earliest_age_diagnosed_GP',
 'DR_inpatient_diagnosis',
 'DR_earliest_inpatient_date',
 'DR_earliest_age_diagnosed_inpatient')

# Alzheimer's

In [54]:
# ALZD

## !!! NO GP CODES

f6148_codes = []
f20002_codes = [1263]
age_diagnosed_self_report_field = None
read_v2_codes = []
read_v3_codes = []
inpatient_icd10_codes = ["F000","F001","F002","F009","G300","G301","G308","G309"]
inpatient_icd9_codes  = []

extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'ALZD',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
)


Self-report


/tmp/ipykernel_3533055/2480788674.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[outcome_col] = np.nan
/tmp/ipykernel_3533055/2480788674.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[outcome_age_col] = np.nan


Baseline, participants with invalid age in f20009.0: 5
Incident, participants with invalid age in f20009.0: 1
ALZD_self_reported
Undiagnosed    502267
Baseline          124
Incident           28
Name: count, dtype: int64
count    146.000000
mean      59.241781
std       12.459987
min        0.000000
25%       56.825000
50%       61.350000
75%       65.300000
max       78.300000
Name: ALZD_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 0
ALZD_GP_diagnosis
Undiagnosed    502419
Name: count, dtype: int64
count      0
mean     NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: ALZD_earliest_GP_date, dtype: object
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: ALZD_earliest_age_diagnosed_GP, dtype: float64

Inpatient


/tmp/ipykernel_3533055/2729423097.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[gp_dx_col] = 'Undiagnosed'
/tmp/ipykernel_3533055/2729423097.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[gp_date_col] = pd.to_datetime(output_df["f.eid"].map(earliest_date_gp), errors="coerce")
/tmp/ipykernel_3533055/2729423097.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

ALZD_inpatient_diagnosis
Undiagnosed    499654
Incident         2747
Baseline           18
Name: count, dtype: int64
count                             2765
mean     2017-10-09 10:10:22.350813696
min                2003-11-24 00:00:00
25%                2016-03-14 00:00:00
50%                2018-05-27 00:00:00
75%                2019-11-23 00:00:00
max                2021-03-30 00:00:00
Name: ALZD_earliest_inpatient_date, dtype: object
count    2765.000000
mean       73.542857
std         5.138113
min        49.000000
25%        71.000000
50%        74.000000
75%        77.000000
max        83.000000
Name: ALZD_earliest_age_diagnosed_inpatient, dtype: float64

Pooled
ALZD_diagnosis_type
Control     499576
Incident      2707
Baseline       136
Name: count, dtype: int64
count    2841.000000
mean       72.948997
std         6.498699
min         0.000000
25%        70.000000
50%        74.000000
75%        77.000000
max        83.000000
Name: ALZD_earliest_age_diagnosed, dtype: float64

Fi

/tmp/ipykernel_3533055/2729423097.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[prevalence_col] = output_df[pooled_dx_col].isin(['Baseline', 'Incident']).astype(int)
/tmp/ipykernel_3533055/2729423097.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[incident_col] = output_df[pooled_dx_col].eq('Incident').astype(int)
/tmp/ipykernel_3533055/2729423097.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

('ALZD_diagnosis_type',
 'ALZD_earliest_age_diagnosed',
 'ALZD_self_reported',
 'ALZD_earliest_age_diagnosed_self_report',
 'ALZD_GP_diagnosis',
 'ALZD_earliest_GP_date',
 'ALZD_earliest_age_diagnosed_GP',
 'ALZD_inpatient_diagnosis',
 'ALZD_earliest_inpatient_date',
 'ALZD_earliest_age_diagnosed_inpatient')

# Parkinson's disease

In [55]:
# PKD

## !!! NO GP CODES

f6148_codes = []
f20002_codes = [1262]
age_diagnosed_self_report_field = None
read_v2_codes = []
read_v3_codes = []
inpatient_icd10_codes = ["G20","G22","G259","G903"]
inpatient_icd9_codes  = ["3320","3321"]

extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'PD',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
)


Self-report


/tmp/ipykernel_3533055/2480788674.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[outcome_col] = np.nan
/tmp/ipykernel_3533055/2480788674.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[outcome_age_col] = np.nan


Baseline, participants with invalid age in f20009.0: 0
Incident, participants with invalid age in f20009.0: 0
PD_self_reported
Undiagnosed    501471
Baseline          857
Incident           91
Name: count, dtype: int64
count    948.000000
mean      58.163502
std        7.473754
min       25.400000
25%       54.175000
50%       59.400000
75%       63.200000
max       77.900000
Name: PD_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 0
PD_GP_diagnosis
Undiagnosed    502419
Name: count, dtype: int64
count      0
mean     NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: PD_earliest_GP_date, dtype: object
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: PD_earliest_age_diagnosed_GP, dtype: float64

Inpatient


/tmp/ipykernel_3533055/2729423097.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[gp_dx_col] = 'Undiagnosed'
/tmp/ipykernel_3533055/2729423097.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[gp_date_col] = pd.to_datetime(output_df["f.eid"].map(earliest_date_gp), errors="coerce")
/tmp/ipykernel_3533055/2729423097.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

PD_inpatient_diagnosis
Undiagnosed    498881
Incident         3143
Baseline          395
Name: count, dtype: int64
count                             3538
mean     2015-04-05 02:22:51.622384896
min                1986-01-29 00:00:00
25%                2012-08-21 00:00:00
50%                2016-05-30 12:00:00
75%                2019-01-15 18:00:00
max                2021-03-30 00:00:00
Name: PD_earliest_inpatient_date, dtype: object
count    3538.000000
mean       69.093556
std         7.468491
min        31.000000
25%        65.000000
50%        70.000000
75%        75.000000
max        83.000000
Name: PD_earliest_age_diagnosed_inpatient, dtype: float64

Pooled
PD_diagnosis_type
Control     498769
Incident      2720
Baseline       930
Name: count, dtype: int64
count    3650.000000
mean       67.475507
std         8.895307
min        25.400000
25%        62.000000
50%        69.000000
75%        74.000000
max        83.000000
Name: PD_earliest_age_diagnosed, dtype: float64

Final defini

/tmp/ipykernel_3533055/2729423097.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[prevalence_col] = output_df[pooled_dx_col].isin(['Baseline', 'Incident']).astype(int)
/tmp/ipykernel_3533055/2729423097.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[incident_col] = output_df[pooled_dx_col].eq('Incident').astype(int)
/tmp/ipykernel_3533055/2729423097.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

('PD_diagnosis_type',
 'PD_earliest_age_diagnosed',
 'PD_self_reported',
 'PD_earliest_age_diagnosed_self_report',
 'PD_GP_diagnosis',
 'PD_earliest_GP_date',
 'PD_earliest_age_diagnosed_GP',
 'PD_inpatient_diagnosis',
 'PD_earliest_inpatient_date',
 'PD_earliest_age_diagnosed_inpatient')

# Multiple sclerosis

In [56]:
# MS

## !!! NO GP CODES

f6148_codes = []
f20002_codes = [1261]
age_diagnosed_self_report_field = None
read_v2_codes = []
read_v3_codes = []
inpatient_icd10_codes = ["G35","G379"]
inpatient_icd9_codes  = ["3409"]

extract_disease_outcomes(
    input_df = ukb49508_df,
    output_df = diagnosis_df,
    col_name = 'MS',
    self_report_data={'positive_vals_f6148': f6148_codes, 'age_diagnosed_field': age_diagnosed_self_report_field, 'positive_vals_f20002': f20002_codes},
    gp_data={'read_v2_codes': read_v2_codes, 'read_v3_codes': read_v3_codes},
    inpatient_data={'icd10_codes': inpatient_icd10_codes, 'icd9_codes': inpatient_icd9_codes},
)


Self-report


/tmp/ipykernel_3533055/2480788674.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[outcome_col] = np.nan
/tmp/ipykernel_3533055/2480788674.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[outcome_age_col] = np.nan


Baseline, participants with invalid age in f20009.0: 4
Incident, participants with invalid age in f20009.0: 0
MS_self_reported
Undiagnosed    500588
Baseline         1777
Incident           54
Name: count, dtype: int64
count    1827.000000
mean       40.938971
std         9.989233
min         0.500000
25%        33.550000
50%        41.100000
75%        47.900000
max        69.300000
Name: MS_earliest_age_diagnosed_self_report, dtype: float64

GP
Participants with invalid age for Dx in GP records: 0
MS_GP_diagnosis
Undiagnosed    502419
Name: count, dtype: int64
count      0
mean     NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: MS_earliest_GP_date, dtype: object
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: MS_earliest_age_diagnosed_GP, dtype: float64

Inpatient


/tmp/ipykernel_3533055/2729423097.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[gp_dx_col] = 'Undiagnosed'
/tmp/ipykernel_3533055/2729423097.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[gp_date_col] = pd.to_datetime(output_df["f.eid"].map(earliest_date_gp), errors="coerce")
/tmp/ipykernel_3533055/2729423097.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

MS_inpatient_diagnosis
Undiagnosed    500178
Baseline         1183
Incident         1058
Name: count, dtype: int64
count                             2241
mean     2008-04-18 21:40:33.734940416
min                1982-01-27 00:00:00
25%                2002-02-27 00:00:00
50%                2008-05-19 00:00:00
75%                2014-05-02 00:00:00
max                2021-03-26 00:00:00
Name: MS_earliest_inpatient_date, dtype: object
count    2241.000000
mean       54.352075
std        10.264433
min        25.000000
25%        47.000000
50%        54.000000
75%        62.000000
max        80.000000
Name: MS_earliest_age_diagnosed_inpatient, dtype: float64

Pooled
MS_diagnosis_type
Control     499878
Baseline      1980
Incident       561
Name: count, dtype: int64
count    2541.000000
mean       45.521133
std        12.477631
min         0.500000
25%        36.800000
50%        45.100000
75%        54.000000
max        80.000000
Name: MS_earliest_age_diagnosed, dtype: float64

Final defini

/tmp/ipykernel_3533055/2729423097.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[prevalence_col] = output_df[pooled_dx_col].isin(['Baseline', 'Incident']).astype(int)
/tmp/ipykernel_3533055/2729423097.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[incident_col] = output_df[pooled_dx_col].eq('Incident').astype(int)
/tmp/ipykernel_3533055/2729423097.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

('MS_diagnosis_type',
 'MS_earliest_age_diagnosed',
 'MS_self_reported',
 'MS_earliest_age_diagnosed_self_report',
 'MS_GP_diagnosis',
 'MS_earliest_GP_date',
 'MS_earliest_age_diagnosed_GP',
 'MS_inpatient_diagnosis',
 'MS_earliest_inpatient_date',
 'MS_earliest_age_diagnosed_inpatient')

# Subcohort definitions

## IOP

In [57]:
# IOP available subcohort = IOP data available & no minor exclusions
# Plus ethnicity and TDI avialable
# Within this ---> glaucoma analysis (exclude controls with exclusion codes), AMD analysis, DR analysis, Cat analysis

IOP_available_mask = ukb51745_df[['f.5262.0.0', 'f.5254.0.0']].isna().all(axis=1) == False
diagnosis_df['IOP_available'] = IOP_available_mask.astype(int)
excluded_mask = diagnosis_df['Prevalent Glaucoma minor exclusions'].isna()

# Exclude individuals with no ethnicity data
missing_ethnicity_mask = ukb49508_df['f.21000.0.0'].isna() | ukb49508_df['f.21000.0.0'].isin([-1, -3])

# Exclude individuals with no TDI
missing_deprivation_mask = ukb675501_df['f.22189.0.0'].isna()

IOP_subcohort_mask = IOP_available_mask & ~excluded_mask & ~missing_ethnicity_mask & ~missing_deprivation_mask

diagnosis_df['IOP_subcohort'] = IOP_subcohort_mask.astype(int)

In [58]:
diagnosis_df['Glaucoma_ML_cohort_control_exclusions'] = ((diagnosis_df['IOP_subcohort'] == 1) & (diagnosis_df['Prevalent Glaucoma control exclusions'].notna())).astype(int)

/tmp/ipykernel_3533055/3555332730.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['Glaucoma_ML_cohort_control_exclusions'] = ((diagnosis_df['IOP_subcohort'] == 1) & (diagnosis_df['Prevalent Glaucoma control exclusions'].notna())).astype(int)


## GP records available

In [59]:
GP_available = diagnosis_df['f.eid'].isin(merged_gp_events_df['f.eid'])
diagnosis_df['GP records available'] = GP_available.astype(int)

/tmp/ipykernel_3533055/3481808052.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df['GP records available'] = GP_available.astype(int)


# Training / test splits

In [60]:
def stratify_split(df, mask_dict, test_proportion, split_col_name, random_state=2024):
    for name, mask in mask_dict.items():
        subgroup_df = df[mask]
        subgroup_test = subgroup_df.sample(frac=test_proportion, replace=False, random_state=random_state)
        df.loc[subgroup_test.index.values, split_col_name] = 'test'
        df.loc[subgroup_df.index.difference(subgroup_test.index.values), split_col_name] = 'train'

In [61]:
def qc_split(df, mask_dict, split_col_name):
    test_proportion = (df[split_col_name] == 'test').sum() / len(df)
    print(f'Split col NA: {df[split_col_name].isna().sum()}')
    print(f'Test set proportion: {test_proportion}')
    print(f'Split col value counts: {df[split_col_name].value_counts()}')
    for name, mask in mask_dict.items():
        test_proportion = (df[mask][split_col_name] == 'test').sum() / len(df[mask])
        print(f'\n {name}')
        print(f'Test set proportion: {test_proportion}')
        print(df[mask][split_col_name].value_counts())

In [62]:
#### Glaucoma
### 80/20 split

split_col = 'training_test_split_glaucoma_80_20_dec2025'
diagnosis_df[split_col] = np.nan

glaucoma_mask = diagnosis_df['Prevalent Glaucoma control exclusions'] == 1
control_mask = diagnosis_df['Prevalent Glaucoma control exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Glaucoma': IOP_subcohort_mask & glaucoma_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

/tmp/ipykernel_3533055/1330865679.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


Split col NA: 391252
Test set proportion: 0.044251909262985674
Split col value counts: training_test_split_glaucoma_80_20_dec2025
train    88934
test     22233
Name: count, dtype: int64

 Glaucoma
Test set proportion: 0.2
training_test_split_glaucoma_80_20_dec2025
train    2956
test      739
Name: count, dtype: int64

 Control
Test set proportion: 0.19999627810034243
training_test_split_glaucoma_80_20_dec2025
train    85978
test     21494
Name: count, dtype: int64


In [63]:
#### AMD
### 80/20 split

split_col = 'training_test_split_amd_80_20_dec2025'
diagnosis_df[split_col] = np.nan

case_mask = diagnosis_df['Prevalent AMD minor exclusions'] == 1
control_mask = diagnosis_df['Prevalent AMD minor exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Case': IOP_subcohort_mask & case_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

/tmp/ipykernel_3533055/292373717.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


Split col NA: 390266
Test set proportion: 0.0446460026392314
Split col value counts: training_test_split_amd_80_20_dec2025
train    89722
test     22431
Name: count, dtype: int64

 Case
Test set proportion: 0.2001196530062818
training_test_split_amd_80_20_dec2025
train    2674
test      669
Name: count, dtype: int64

 Control
Test set proportion: 0.2
training_test_split_amd_80_20_dec2025
train    87048
test     21762
Name: count, dtype: int64


In [64]:
#### Cataract
### 80/20 split

split_col = 'training_test_split_cataract_80_20_dec2025'
diagnosis_df[split_col] = np.nan

case_mask = diagnosis_df['Prevalent Cataract minor exclusions'] == 1
control_mask = diagnosis_df['Prevalent Cataract minor exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Case': IOP_subcohort_mask & case_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

/tmp/ipykernel_3533055/3884677106.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


Split col NA: 390266
Test set proportion: 0.0446460026392314
Split col value counts: training_test_split_cataract_80_20_dec2025
train    89722
test     22431
Name: count, dtype: int64

 Case
Test set proportion: 0.2000254161901131
training_test_split_cataract_80_20_dec2025
train    12590
test      3148
Name: count, dtype: int64

 Control
Test set proportion: 0.2
training_test_split_cataract_80_20_dec2025
train    77132
test     19283
Name: count, dtype: int64


In [65]:
#### DR
### 80/20 split
# diabetes cohort only

diabetes_cohort_mask = diagnosis_df['Prevalent T1orT2DM minor exclusions'] == 1

split_col = 'training_test_split_dr_80_20_dec2025'
diagnosis_df[split_col] = np.nan

case_mask = diagnosis_df['Prevalent DR minor exclusions'] == 1
control_mask = diagnosis_df['Prevalent DR minor exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Case': IOP_subcohort_mask & diabetes_cohort_mask & case_mask,
    'Control': IOP_subcohort_mask & diabetes_cohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

Split col NA: 492068
Test set proportion: 0.004122057485883296
Split col value counts: training_test_split_dr_80_20_dec2025
train    8280
test     2071
Name: count, dtype: int64

 Case
Test set proportion: 0.20014609203798392
training_test_split_dr_80_20_dec2025
train    2190
test      548
Name: count, dtype: int64

 Control
Test set proportion: 0.20005254170497833
training_test_split_dr_80_20_dec2025
train    6090
test     1523
Name: count, dtype: int64


/tmp/ipykernel_3533055/2509188010.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


In [66]:
#### ALZD
### 80/20 split

split_col = 'training_test_split_alzd_80_20_dec2025'
diagnosis_df[split_col] = np.nan

case_mask = diagnosis_df['Prevalent ALZD minor exclusions'] == 1
control_mask = diagnosis_df['Prevalent ALZD minor exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Case': IOP_subcohort_mask & case_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

/tmp/ipykernel_3533055/1948808987.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


Split col NA: 390266
Test set proportion: 0.0446460026392314
Split col value counts: training_test_split_alzd_80_20_dec2025
train    89722
test     22431
Name: count, dtype: int64

 Case
Test set proportion: 0.20077972709551656
training_test_split_alzd_80_20_dec2025
train    410
test     103
Name: count, dtype: int64

 Control
Test set proportion: 0.2
training_test_split_alzd_80_20_dec2025
train    89312
test     22328
Name: count, dtype: int64


In [67]:
#### PD
### 80/20 split

split_col = 'training_test_split_pd_80_20_dec2025'
diagnosis_df[split_col] = np.nan

case_mask = diagnosis_df['Prevalent PD minor exclusions'] == 1
control_mask = diagnosis_df['Prevalent PD minor exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Case': IOP_subcohort_mask & case_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

/tmp/ipykernel_3533055/3492053087.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


Split col NA: 390266
Test set proportion: 0.0446440122686443
Split col value counts: training_test_split_pd_80_20_dec2025
train    89723
test     22430
Name: count, dtype: int64

 Case
Test set proportion: 0.19973009446693657
training_test_split_pd_80_20_dec2025
train    593
test     148
Name: count, dtype: int64

 Control
Test set proportion: 0.19999640972247154
training_test_split_pd_80_20_dec2025
train    89130
test     22282
Name: count, dtype: int64


In [68]:
#### MS
### 80/20 split

split_col = 'training_test_split_ms_80_20_dec2025'
diagnosis_df[split_col] = np.nan

case_mask = diagnosis_df['Prevalent MS minor exclusions'] == 1
control_mask = diagnosis_df['Prevalent MS minor exclusions'] == 0

# IOP subcohort only
mask_dict = {
    'Case': IOP_subcohort_mask & case_mask,
    'Control': IOP_subcohort_mask & control_mask,
}
stratify_split(
    df=diagnosis_df,
    mask_dict=mask_dict,
    test_proportion=0.2,
    split_col_name=split_col,
    random_state=2024,
)
qc_split(diagnosis_df, mask_dict, split_col)

/tmp/ipykernel_3533055/3651917864.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diagnosis_df[split_col] = np.nan
/tmp/ipykernel_3533055/1883662268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'test' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[subgroup_test.index.values, split_col_name] = 'test'


Split col NA: 390266
Test set proportion: 0.0446460026392314
Split col value counts: training_test_split_ms_80_20_dec2025
train    89722
test     22431
Name: count, dtype: int64

 Case
Test set proportion: 0.20077972709551656
training_test_split_ms_80_20_dec2025
train    410
test     103
Name: count, dtype: int64

 Control
Test set proportion: 0.2
training_test_split_ms_80_20_dec2025
train    89312
test     22328
Name: count, dtype: int64


# Export

In [80]:
exclude_cols = ['DOB_est', 'Ever pregnant', 'Sex_nondisc']
cols_to_save = diagnosis_df.columns[~diagnosis_df.columns.isin(exclude_cols)].tolist()

In [ ]:
# cols_to_save = [
#     'f.eid',
#     'IOP_available', 'IOP_subcohort',
#     'Prevalent Glaucoma minor exclusions', 'Incident Glaucoma minor exclusions', 'Baseline Glaucoma minor exclusions',
#     'Prevalent Glaucoma control exclusions', 'Incident Glaucoma control exclusions', 'Baseline Glaucoma control exclusions',
#     'Glaucoma_earliest_age_diagnosed',
#     'Prevalent Glaucoma control and age40 exclusions', 'Baseline Glaucoma control and age40 exclusions', 'Incident Glaucoma control and age40 exclusions',
#     'prevalent_glaucoma_surgery_or_med', 'baseline_glaucoma_surgery_or_med', 'incident_glaucoma_surgery_or_med',
#     'prevalent_glaucoma_surgery', 'baseline_glaucoma_surgery', 'incident_glaucoma_surgery',
#     'prevalent_glaucoma_med', 'baseline_glaucoma_med', 'incident_glaucoma_med',
    
#     'Prevalent AMD minor exclusions', 'Incident AMD minor exclusions', 'Baseline AMD minor exclusions',
#     'Prevalent Cataract minor exclusions', 'Incident Cataract minor exclusions', 'Baseline Cataract minor exclusions',
#     'Prevalent T1orT2DM minor exclusions', 'Incident T1orT2DM minor exclusions', 'Baseline T1orT2DM minor exclusions',
#     'Prevalent DR minor exclusions', 'Incident DR minor exclusions', 'Baseline DR minor exclusions',
    
#     'Prevalent ALZD minor exclusions', 'Incident ALZD minor exclusions', 'Baseline ALZD minor exclusions',
#     'Prevalent PD minor exclusions', 'Incident PD minor exclusions', 'Baseline PD minor exclusions',
#     'Prevalent MS minor exclusions', 'Incident MS minor exclusions', 'Baseline MS minor exclusions',

#     'GP records available',
#     'Glaucoma_ML_cohort_control_exclusions',
    
#     'training_test_split_glaucoma_80_20_dec2025', 
#     'training_test_split_amd_80_20_dec2025', 
#     'training_test_split_cataract_80_20_dec2025',
#     'training_test_split_dr_80_20_dec2025',
#     'training_test_split_pd_80_20_dec2025',
#     'training_test_split_alzd_80_20_dec2025',
#     'training_test_split_ms_80_20_dec2025',
# ]

In [84]:
diagnosis_df[cols_to_save].to_pickle('./data/extracted_outcome_definitions.pkl', compression=None)

In [85]:
diagnosis_df[cols_to_save].to_csv('./data/extracted_outcome_definitions.csv', sep='\t', index=False)